In [1]:
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from collections import Counter
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from re import finditer,sub,search
import os, math, random
from colorama import Back, Style, Fore

# For blastwww

from Bio import SeqIO, SearchIO
from Bio.Blast import NCBIWWW 
from Bio.Seq import Seq
import pandas as pd
import panel as pn
import panel.widgets as pnw
pn.extension()
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, Plot, Grid, Range1d
from bokeh.models.glyphs import Text, Rect
from bokeh.layouts import gridplot
from bokeh.transform import dodge
import bokeh

''' Display List '''
# display list neatly
# https://stackoverflow.com/questions/1524126/how-to-print-a-list-more-nicely
def lstcol(obj, cols=4, columnwise=True, gap=4):
    sobj = [str(item) for item in obj]
    if cols > len(sobj): cols = len(sobj)
    max_len = max([len(item) for item in sobj])
    if columnwise: cols = int(math.ceil(float(len(sobj)) / float(cols)))
    plist = [sobj[i: i+cols] for i in range(0, len(sobj), cols)]
    if columnwise:
        if not len(plist[-1]) == cols:
            plist[-1].extend(['']*(len(sobj) - len(plist[-1])))
        plist = zip(*plist)
    printer ='\n'.join([
        ''.join([c.ljust(max_len + gap) for c in p])
        for p in plist])
    print (printer)

In [2]:
from IPython.core.display import display, HTML, Javascript

color_map = ['#FFFFFF','#FF5733']

prompt = color_map[-1]
main_color = color_map[0]
strong_main_color = color_map[1]
custom_colors = [strong_main_color, main_color]

css_file = '''
div #notebook {
background-color: white;
line-height: 20px;
}

#notebook-container {
%s
margin-top: 2em;
padding-top: 2em;
border-top: 4px solid %s;
-webkit-box-shadow: 0px 0px 8px 2px rgba(224, 212, 226, 0.5);
    box-shadow: 0px 0px 8px 2px rgba(224, 212, 226, 0.5);
}

div .input {
margin-bottom: 1em;
}

.rendered_html h1, .rendered_html h2, .rendered_html h3, .rendered_html h4, .rendered_html h5, .rendered_html h6 {
color: %s;
font-weight: 600;
}

div.input_area {
border: none;
    background-color: %s;
    border-top: 2px solid %s;
}

div.input_prompt {
color: %s;
}

div.output_prompt {
color: %s; 
}

div.cell.selected:before, div.cell.selected.jupyter-soft-selected:before {
background: %s;
}

div.cell.selected, div.cell.selected.jupyter-soft-selected {
    border-color: %s;
}

.edit_mode div.cell.selected:before {
background: %s;
}

.edit_mode div.cell.selected {
border-color: %s;

}
'''

def to_rgb(h): 
    return tuple(int(h[i:i+2], 16) for i in [0, 2, 4])

main_color_rgba = 'rgba(%s, %s, %s, 0.1)' % (to_rgb(main_color[1:]))
open('notebook.css', 'w').write(css_file % ('width: 95%;', main_color, main_color, main_color_rgba, 
                                            main_color,  main_color, prompt, main_color, main_color, 
                                            main_color, main_color))

def nb(): 
    return HTML("<style>" + open("notebook.css", "r").read() + "</style>")
nb()

![](https://i.imgur.com/1PtktjN.png)

#### <b><span style='color:#F1C40F'>NOTEBOOK AIM</span></b>

- Whilst there are wonderful libraries like **<span style='color:#F1C40F'>BioPython</span>** that allow us to work with **<span style='color:#F1C40F'>biological sequences</span>**
- It's often quite benefitial to understand the workings of the code & expand on the code if a need arises, in this notebook, we use a very simple class structure
- This is more time consuming, nevertheless it's definitely more interesting & allows us to incorporate different libraries (eg.**<mark style="background-color:#212121;color:white;border-radius:5px;opacity:0.8">bokeh</mark>**) 
- In this notebook, emphasis will be placed on replicating something similar to two BioPython modules:
> - from Bio import **<mark style="background-color:#F1C40F;color:white;border-radius:5px;opacity:0.9">SeqIO</mark>**  ( Class for readng sequences )
> - from Bio.Seq import **<mark style="background-color:#F1C40F;color:white;border-radius:5px;opacity:0.9">Seq</mark>** ( Class for Sequence Operations )


- At the very end of this notebook, we'll also create a **<mark style="background-color:#212121;color:white;border-radius:5px;opacity:0.8">whl</mark>** package, which is uploaded & updated in the dataset: **[bioseq](https://www.kaggle.com/shtrausslearning/bioseq)**, which includes classes used here
- **<mark style="background-color:#212121;color:white;border-radius:5px;opacity:0.8">whl</mark>** is the format of packages used on pip & these files can be uploaded to datasets, installed via pip & called as you would an existing pip package on Kaggle
- If you are interested in bioinformatics & I'd definitely recommend getting to know **[bioconductor](https://www.bioconductor.org/)** as well, I've made a simple introductory notebook **[bioconductor - bioinformatics basics](https://www.kaggle.com/shtrausslearning/bioconductor-bioinformatics-basics)** as well

# <div style="padding: 30px;color:white;margin:10;font-size:80%;text-align:left;display:fill;border-radius:10px;background-color:#F1C40F;overflow:hidden;background-image: url(https://i.imgur.com/tNNAhbu.png)"><b><span style='color:white'>1 |</span></b> Background</div>

<div style="color:white;display:fill;border-radius:8px;
            background-color:#03112A;font-size:150%;
            letter-spacing:1.0px">
    <p style="padding: 8px;color:white;"><b><b><span style='color:#F1A424'>1.1 |</span></b> Section Overview</b></p>
</div>

In this section, we'll introduce some key terminology associated with what we will look into in this notebook:

- **<mark style="background-color:#212121;color:white;border-radius:5px;opacity:0.8">1.2</mark>** - We'll introduce information about biological cells
- **<mark style="background-color:#212121;color:white;border-radius:5px;opacity:0.8">1.3</mark>** - We'll introduce the two commonly used alphabets (neucleotides & amino acids)
- **<mark style="background-color:#212121;color:white;border-radius:5px;opacity:0.8">1.4</mark>** - DNA is made up of two strands, we'll mention a couple of words about them
- **<mark style="background-color:#212121;color:white;border-radius:5px;opacity:0.8">1.5</mark>** - In this section we'll introduce two main sythesis methods (**<span style='color:#F1C40F'>RNA</span>** & **<span style='color:#F1C40F'>Protein</span>**) 
- **<mark style="background-color:#212121;color:white;border-radius:5px;opacity:0.8">1.6</mark>** - We'll talk about what we'll be making in this notebook, in order to do some basic **biological seqence operations**

<div style="color:white;display:fill;border-radius:8px;
            background-color:#03112A;font-size:150%;
            letter-spacing:1.0px">
    <p style="padding: 8px;color:white;"><b><b><span style='color:#F1A424'>1.2 |</span></b> Cell Information</b></p>
</div>

Cells contain molecules which will reference to througout the notebook: **<mark style="background-color:#F1C40F;color:white;border-radius:5px;opacity:0.9">amino acids</mark>**, **<mark style="background-color:#F1C40F;color:white;border-radius:5px;opacity:0.9">nucleotides</mark>**, **<mark style="background-color:#F1C40F;color:white;border-radius:5px;opacity:0.9">proteins</mark>**

A **<mark style="background-color:#F1C40F;color:white;border-radius:5px;opacity:0.9">cell</mark>** is mostly composed of water:
> - a bacteria cell has a weight composition of roughly 70% water & 30% <b>chemical origin</b>, 
> - <b>7% -> small molecules</b> Inc. **<mark style="background-color:#F1C40F;color:white;border-radius:5px;opacity:0.9">amino acids</mark>** and **<mark style="background-color:#F1C40F;color:white;border-radius:5px;opacity:0.9">nucleotides</mark>**
> - 23% -> <b>macro molecules</b> (**<mark style="background-color:#F1C40F;color:white;border-radius:5px;opacity:0.9">proteins</mark>**,lipids,polysaccharides)



According to their internal structure, they can be divided into to major categories:

> - **<mark style="background-color:#F1C40F;color:white;border-radius:5px;opacity:0.9">Prokaryotic</mark>** cells : have no nucleus or internal membranes
> - **<mark style="background-color:#F1C40F;color:white;border-radius:5px;opacity:0.9">Eukaryotic</mark>** cells : which have a defined <b>nucleus</b>, <b>internal membranes</b> and functional elements called <b>organelles</b>.


- At a structural level, all cells are surrounded by a structure called cell <b>membrane</b> or <b>plasma membrane</b>. 
- This <b>membrane</b> is permeable to molecules that cells need to absorb from or excrete to the outside medium.
- Within the cell we find the <b>cytoplasm</b> (largely composed of water), which serves as the medium for the cell

<div style="color:white;display:fill;border-radius:8px;
            background-color:#03112A;font-size:150%;
            letter-spacing:1.0px">
    <p style="padding: 8px;color:white;"><b><b><span style='color:#F1A424'>1.3 |</span></b> Sequence Alphabets</b></p>
</div>

**<mark style="background-color:#323232;color:white;border-radius:5px;opacity:0.9">ABC (I/II)</mark>** **<mark style="background-color:#F1C40F;color:white;border-radius:5px;opacity:0.9">Nucleic Acids</mark>**

> - Among <b>molecules with a biological role</b>, we can find **<span style='color:#F1C40F'>nucleic acids</span>**
> - Nucleic acids encode and express the genetic code that is kept within the cell
- There are two major types of **<span style='color:#F1C40F'>nucleic acids</span>**: 
> - **<span style='color:#F1C40F'>Deoxyribo Nucleic Acid (DNA)</span>**
> - **<span style='color:#F1C40F'>Ribonucleic Acid (RNA)</span>** (Obtainable via transcription)
- DNA contains the information necessary to build a cell, and keep it functioning. 
- In <b>eukaryotic</b> cells, DNA will be found in the nucleus, whilst in the <b>prokaryotic</b> cells, it will be found in the cytoplasm. 
- <b>IUPAC</b> defines the full list of nucleotides as shown in the table below, with <b>A,T,G,C</b> being the main four:
- Another type of nucleotide list often used is **[IUB Ambiguity Codes](http://biocorp.ca/IUB.php)**, which we use later in the notebook as well

**<mark style="background-color:#323232;color:white;border-radius:5px;opacity:0.9">ABC (II/II)</mark>** **<mark style="background-color:#F1C40F;color:white;border-radius:5px;opacity:0.9">Amino Acids</mark>**

**<span style='color:#F1C40F'>Amino acids</span>**:
> The **building blocks of proteins**, which are <b>macromolecules</b> that perform most of the functions inside a cell

Proteins have a **broad range of functions**, spanning from **catalytic** to **structural functions**:

> - **<span style='color:#F1C40F'>Enzymes</span>** : Type of abundant proteins that promote chemical reactions & convert some molecules into other types of molecules required for the functioning of the cell
> - **<span style='color:#F1C40F'>Carbohydrates</span>** : Serve as energy storage, both for immediate and long term energy demands
> - **<span style='color:#F1C40F'>Lipids</span>** : Part of the plasma membrane, doing signaling and energy storage

The cell also contains other components of varying complexity. Of importance: 
> - <b>Mitochondria</b> & the <b>Chloroplasts</b> : Organelles involved in the production of energy. 
> - <b>Ribosomes</b> : Large and complex molecules composed of a mixture of genetic material, req. to assemble proteins and play a central role in the flow of genetic information


<div style="color:white;display:fill;border-radius:8px;
            background-color:#03112A;font-size:150%;
            letter-spacing:1.0px">
    <p style="padding: 8px;color:white;"><b><b><span style='color:#F1A424'>1.4 |</span></b> DNA Strands</b></p>
</div>

#### <b><span style='color:#F1C40F'>COMPLEMENTARY STRANDS IN DNA</span></b>

- DNA is a molecule composed of **<span style='color:#F1C40F'>two complementary strands</span>** that form and stick together due to the connections established between the nucleotides in both strands
- This is made possible by due to the chemical phenomenon:
    - Where **<mark style="background-color:#F1C40F;color:white;border-radius:5px;opacity:0.9">Adenine (A)</mark>** bonds only with **<mark style="background-color:#F1C40F;color:white;border-radius:5px;opacity:0.9">Thymine (T)</mark>** nucleotides; as a result of **two hydrogen connections**
    - Similarly, **<mark style="background-color:#F1C40F;color:white;border-radius:5px;opacity:0.9">Guanine (G)</mark>** bonds only with **<mark style="background-color:#F1C40F;color:white;border-radius:5px;opacity:0.9">Cytosine (C)</mark>** nucleotides by three hydrogen connections

#### <b><span style='color:#F1C40F'>REVERSE COMPLEMENT</span></b>

- This results in **two complementary** and **anti-parallel strands** (connected in opposite directions), if we know the nucleotide sequence in one of the strands, we can get the sequence in the opposite strand by taking the complement of its nucleotides, which are also read backwards, thus we have the **reverse complement** of the other strand.
- It has become a **standard to describe the DNA though only one** of the strands, due to this **complementarity** using <b>[A,T,G,C]</b>.
- The existence of these two strands is essential in order to **pass on genetic information** to new cells and **produce proteins**.

<div style="color:white;display:fill;border-radius:8px;
            background-color:#03112A;font-size:150%;
            letter-spacing:1.0px">
    <p style="padding: 8px;color:white;"><b><b><span style='color:#F1A424'>1.5 |</span></b> Central Dogma of Molecular & Cell Biology</b></p>
</div>

- The process talked about in the next section is quite complex & not all aspects are competely understood,  a general and simplistic picture is provided below
- **<span style='color:#F1C40F'>DNA</span>**, **<span style='color:#F1C40F'>RNA</span>** & **<span style='color:#F1C40F'>Proteins</span>** are the central elements of the flow of genetic information that occurs in two steps:

**<mark style="background-color:#212121;color:white;border-radius:5px;opacity:0.8">STEP 1</mark>** <b>RNA SYNTHESIS : <span style='color:#F1C40F'>TRANSCRIPTION</span></b>

- Preliminary step required to produce a <b>protein</b>
- The nucleotide sequence of a gene from one of the DNA strands is transcribed ( copied into a complementary molecule of RNA )
- The complementarity of the genetic code allows recovering the information encoded in the original DNA sequence, a process performed by the enzyme, RNA polymerase
- Additional steps of RNA processing, including stabilising elements at the end of the molecule, are performed by different protein complexes
- After these steps, which occur within the nucleus of the cell, an RNA molecule; <b>mature messenger RNA (mRNA)</b> is obtained.
- The <b>mRNA</b> is then transported to the cytoplasm, where it will be used by the cellular machine to guide the production of a protein

**<mark style="background-color:#212121;color:white;border-radius:5px;opacity:0.8">STEP 2</mark>** <b>PROTEIN SYNTHESIS : <span style='color:#F1C40F'>TRANSLATION</span></b>

- Process in which the nucleotide sequence of the mRNA is <b>transcribed into a chain of amino acids</b>, forming a <b>polypeptide</b>. 
- <b>Proteins</b> are cellular entities that have either:
  - (a) **<span style='color:#F1C40F'>Structural function</span>** - Participating in the physical definition of a cell.
  - (b) **<span style='color:#F1C40F'>Chemical function</span>** - Being involved in chemical reactions occuring in the cell.
-  In order to function as expected, a protein needs to acquire the appropriate structure & this structure is often decomposed at different complexity levels

The primary structure is defined by the chain of amino acids; <b>polypeptide</b>, consisting in part or completely of protein.
- This process is performed by the **ribosomes** that attach and scan the mRNA from one end to the other, in groups of <b>nucleotide triplets</b> / <b>Codons</b>.

**<mark style="background-color:#212121;color:white;border-radius:5px;opacity:0.8">CODONS</mark>**
    
- In each position of the triplet, we have 1/4 nucleotides, ie. there are 4x4x4 (64) possible triplets/Codons.
- For each codon in the mRNA sequence, we have a corresponding amino acid in the <b>polypeptide chain</b>
    
**<mark style="background-color:#212121;color:white;border-radius:5px;opacity:0.8">START AND STOP CODON</mark>**
    
- Some of these codons represent significant signals that indicate the <b>initiation</b> or the <b>termination</b> of the translation process.
- Once the **ribosome** detects an initiation codon, it starts the formation of the amino acid chain, 
- When it scans the stop codon, it stops the translation and detaches from the mRNA molecule
- There are <b>20 types of amino acids used to form polypeptides</b> (for IUPAC) & less than the 64 possible codons, therefore we have more than one codon corresponds to a type of amino acid
- During the translation process:
  - A type of small RNA molecule, **<span style='color:#F1C40F'>transfer RNA (tRNAs)</span>** will bring to the <b>ribosome</b>, the amino acids of the corresponding type, which will be complementary to the mRNA codon that is currently being scanned.
  - Each mRNA molecule can be scanned multiple times by different ribosomes, giving rise to multiple copies of the polypeptide. 
  - With its redundency, where more than one codon encodes an amino acid, the genetic code encloses a very efficient code-correction mechanism that minimises the impact of errors in the nucleotide sequence occuring in DNA replication.

**<mark style="background-color:#212121;color:white;border-radius:5px;opacity:0.8">OPEN READING FRAMES</mark>**

During translation process:
- Parsing of the **<span style='color:#F1C40F'>mRNA sequence</span>** by the ribosome **<span style='color:#F1C40F'>may start at different nucleotides</span>**. 
- Given that a codon is composed of three nucleotides, the mRNA sequence may have <b>3 possible interpretations</b>. 
- These three ways of parsing the sequence are called **<span style='color:#F1C40F'>reading frames</span>**.

# <div style="padding: 30px;color:white;margin:10;font-size:80%;text-align:left;display:fill;border-radius:10px;background-color:#F1C40F;overflow:hidden;background-image: url(https://i.imgur.com/tNNAhbu.png)"><b><span style='color:white'>2 |</span></b> Sequence Operations Class</div>

- We define a custom class, **<mark style="background-color:#212121;color:white;border-radius:5px;opacity:0.8">SQ</mark>**, which contains basic <b>sequence related operations</b>
- The <b>sequence class</b> **<mark style="background-color:#212121;color:white;border-radius:5px;opacity:0.8">SQ</mark>** can be coupled with other classes that incorporate more sophisticated sequence based operations; eg. **[sequence alignment](https://www.kaggle.com/shtrausslearning/biological-sequence-alignment)**

<div style="color:white;display:fill;border-radius:8px;
            background-color:#03112A;font-size:150%;
            letter-spacing:1.0px">
    <p style="padding: 8px;color:white;"><b><b><span style='color:#F1A424'>CLASS |</span></b> Sequence Class</b></p>
</div>

**<mark style="background-color:#212121;color:white;border-radius:5px;opacity:0.8">ATTRIBUTES</mark>**
- <code>seq</code> : String format of sequence
- <code>seq_type</code> : Sequence Character Type
- <code>id</code> : Sequence Identifier 
- <code>name</code> : used to allocate a name to the sequence
- <code>description</code> : used to show more information about the sequence
- <code>annotation</code> : used to store annotation information about our sequence

**<mark style="background-color:#212121;color:white;border-radius:5px;opacity:0.8">CLASS METHODS</mark>**
- <code>info</code> : Show the sequence & type **<span style='color:#F1C40F'>information only</span>**
- <code>abc</code> : Show the **<span style='color:#F1C40F'>alphabet characters</span>** of the the sequence type
- <code>validate</code> : Determine if the current **<span style='color:#F1C40F'>sequence contains valid subsets</span>** of charaters for its type
- <code>freq</code> : Calculate & plot the **<span style='color:#F1C40F'>frequencey of each character</span>** in the alphabet of the define sequence **<span style='color:#F1C40F'>type</span>**
- <code>count_purines</code> : Count purines  <code>A&G</code> sum & pyrimidines <code>C&T</code>
- <code>groupfreq</code> : Cound dinucleotides & trincucleotide groupings in sequence
- <code>gc</code> : Calculate the **<span style='color:#F1C40F'>GC content</span>** of the sequence
- <code>reverse_comp</code> : Rearrange the DNA sequnce to show its **<span style='color:#F1C40F'>reverse complement sequence pair</span>**
- <code>transcription</code> : Convert DNA sequence into an **<span style='color:#F1C40F'>RNA sequence</span>** 
- <code>get_protein</code> : Derive all the **<span style='color:#F1C40F'>amino acid chains</span>** in **<span style='color:#F1C40F'>all reading frames</span>** and store as one sequence
- <code>find_pattern</code> : Find standard/string patterns in the sequence & more specialised/specific patterns
- <code>cut_pattern</code> : Cut sequence in specific parts based on a pattern

**<mark style="background-color:#212121;color:white;border-radius:5px;opacity:0.8">MAPPING DICTIONARY</mark>**

- Similar to BioPython's <b>CodonTable</b> as shown in **[Biopython for Bioinformatics Basics](https://www.kaggle.com/shtrausslearning/biopython-for-bioinformatics-basics)**
- A dictionary containing mapping data between different combinations of mRNA nucleotides (codons) and <b>amino acids</b> is obtainable in <b>dic_map</b> by setting <b>map_id</b> to *codon*. 
- Similarly, mapping between the IUPAC amino acid symbols & names can be obtained by using <b>map_id = 'iupac_amino'</b> &  <b>map_id = 'iupac_nucleotide'</b> respectively.

In [3]:
'''

#####################################################

# General SQ Methods

# info - show sequence information
# view - visualise the sequence
# validate - check if the sequence contains no errors
# reverse_comp - reverse complement strand of DNA 
# add_annotation - add annotation to sequence

#####################################################

'''

class SQ(): 
    
    # Constructor
    def __init__ (self, seq=None,           # Sequence String
                        seq_type = "dna",   # Sequence Type 
                        id=None,            # Sequence Identifier
                        name=None,          # Sequene Name
                        description=None):  # Sequence Description
        
        # Core SQ 
        self.seq = seq.upper()
        self.seq_type = seq_type
        
        # If more detail is provided
        self.id = id  # sequence identifier (eg. database id)
        self.name = name   # name of sequence (eg. polyprotein)
        self.description = description # give the sequence some description
        self.annotations = {} # Annotate some parts of the seqence
        
        # if some fields are not filled
        if(self.id is None):
            self.id = f'sequence_{random.randint(1,1000)}'
        if(self.description is None):
            self.description = 'description not given'
        if(self.name is None):
            self.name = 'name not given'
        
        # Instantiate 
        self.decode = Decode(self.seq,self.seq_type) 
        self.count = Count(self.seq,self.seq_type,self.id)
        self.cut = Cut(self.seq,self.seq_type)
        self.pattern = Pattern(self.seq,self.seq_type)
    
    # class instance operations
    def __len__(self):
        return len(self.seq)
    def __getitem__(self, n):
        return self.seq[n]
    def __getslice__(self, i, j):
        return self.seq[i:j]
    def __str__(self):
        return self.seq
    def __add__(self,other):
        if(self.seq_type == other.seq_type):
            return SQ(self.seq + other.seq,seq_type=self.seq_type)
        else:
            print('sequences must of be same type')
            
    @staticmethod
    def colored(lseq):
        
        bcolors = {'A': '\033[92m','C': '\033[94m','G':'\033[93m',
            'T': '\033[91m','U': '\033[91m','reset': '\033[0;0m'}
        tmpStr = ""
        for nuc in lseq:
            if nuc in bcolors:
                tmpStr += bcolors[nuc] + nuc
            else:
                tmpStr += bcolors['reset'] + nuc
        return tmpStr + '\033[0;0m'
    
    @staticmethod
    def split_nucleotide_seq(genome,cut_id):
        genes = []
        for ix, char in enumerate(genome):
            if ix != 0 and ix%cut_id == 0:
                genes.append(' ')
            genes.append(char)
        return ''.join(genes)
    
    def seq_repr(self,genome_str, strand ='dna',split_id=10):
        
        nu_clr_switcher = {
            # standard color-codes
            'A': Back.GREEN,
            'a': Back.GREEN,
            'C': Back.YELLOW,
            'c': Back.YELLOW,
            'G': Back.RED,
            'g': Back.RED,
            'T': Back.BLUE,
            't': Back.BLUE,
            ' ': Style.RESET_ALL
        }
        
        if strand == 'dna':
            genome_str = self.split_nucleotide_seq(genome=genome_str,
                                                   cut_id=split_id)
            line_break_cntr = 0
            for i in range(len(genome_str)):
                if genome_str[i] == ' ':
                    line_break_cntr += 1
                    if line_break_cntr>0 and line_break_cntr%6==0:
                        text = "\n"
                    else:
                        text = nu_clr_switcher[genome_str[i]] + genome_str[i]
                else:
                    text = nu_clr_switcher[genome_str[i]] + genome_str[i]
                print(text, end="")
            Style.RESET_ALL
    
    def info(self):
        if(self.seq_type is 'dna' or self.seq_type is 'rna'):
            print (f"SEQ: {self.colored(self.seq)}" + \
                   " "+ f"TYPE: {self.seq_type}")
        else:
            print (f"SEQ: {self.seq}" +" " + \
                   f"TYPE: {self.seq_type}")
    
    # Visualise the nucleotide sequence
            
    def view(self,split_id=10):
        self.seq_repr(self.seq,self.seq_type,split_id)
        
    # Check Validity
    def validate(self,verbose=False):
        alp = self.count.abc()
        res = True; i = 0
        while (res and i < len(self.seq)):
            if self.seq[i] not in alp: 
                res = False
            else: i += 1
        if(res):
            if(verbose):
                print(f'{self.seq_type} is valid')
            return res
        else:
            if(verbose):
                print(f'{self.seq_type} is invalid')
            return res
        
    # Reverse Complement
    def reverse_comp(self):
        
        if (self.seq_type != "dna"): 
            print('input not DNA')
            return None
        
        lst_seq = ['A','T','G','C']
        lst_comp = ['T','A','C','G']
        
        comp = ''
        for char in self.seq:
            ii=-1
            for c in lst_seq:
                ii+=1
                if(char == c ):
                    comp = lst_comp[ii] + comp
                    
        return SQ(comp,"dna")
    
    #####################################################
            
    # Annotating Sequence, Notes
    # add_annotation - add_annotation
            
    #####################################################
    
    # method to add annotation to the current detailed sequence
    def add_annotation(self,inSQ):
        lseq = inSQ 
        lid = inSQ.id
        ldesc = inSQ.description
        idx = self.pattern.find(pattern=str(lseq),
                                   find_id='all',                                            
                                   search_id='standard',
                                   verbose=False)
        for i in idx:
            ln = len(lseq)
            self.annotations[f'sq_{i}:{i+ln}'] = f"{lid}_{ldesc}"

In [4]:
'''

########################################################

Cutting the Sequence
cut_pattern - cut sequence based on particular pattern
dnaseq_features - cut the sequence 

########################################################

'''

class Cut:
    
    # Constructor
    def __init__(self,seq,seq_type):
        self.seq = seq
        self.seq_type = seq_type
            
    # converts IUB ambiguity code into RE
    # returns cut position of a restriction enzyme 
    # (in IUB code) in a sequence
            
    @staticmethod
    def divide_loc(enzyme, sequence):
        
        def iubrex(IUB):   
            
            # main 4 bases
            # purine, pyrimidine, amino
            # keto, strong, weak
            # not A, not C, not G, not T
            #  not T, any 
            
            dic = {"A":"A", "C":"C", "G":"G", "T":"T", 
                    # Additional Cases
                    "R":"[GA]", "Y":"[CT]", "M":"[AC]", 
                    "K":"[GT]", "S":"[GC]", "W": "[AT]",
                    "B":"[CGT]", "D":"[AGT]", "H":"[ACT]",
                    "V":"[ACG]", "N":"[ACGT]"}
            
            site = IUB.replace("|","")
            rex = ""
            
            for c in site:
                rex += dic[c]
            return rex
        
        regexp = iubrex(enzyme) # convert pattern to IUB format
        matches = finditer(regexp, sequence)
        locs = []
        for match in matches:
            locs.append(match.start() + enzyme.find("|"))
            
        return locs # indicies of cuts
    
    # determines subsequences resulting from a sequence 
    # cut in a list of positions
    def cut_pattern(self,cut_pattern=None):
        
        if(cut_pattern is not None):
        
            res = []
            positions = self.divide_loc(cut_pattern,self.seq)
            positions.insert(0,0)
            positions.append(len(self.seq))
            for i in range(len(positions)-1):
                res.append(self.seq[positions[i]:positions[i+1]])
            return res
        
        else:
            print('[note] enter cut_pattern')
    
    # Function for when you want to prepare DNA sequence 
    # feature for ML applications
    def dnaseq_features(self,start=0,n_segs=101,seq_name=None):
        
        print(f'[note] cutting @{start} w/ {n_segs} segments')
        
        print(f"Input Sequence Length: {len(self.seq)}")
        remaind = len(self.seq)%n_segs
        if(remaind is not 0):
            last_id = len(self.seq) - remaind
        print(f"# Bases cut-off: {int(remaind)}")
        
        upd_seq = self.seq[start:last_id]
        
        print(f"Updated sequence length: {len(upd_seq)}")
        print(f"# Segments: {int(len(upd_seq)/n_segs)} created")
        if(seq_name is None):
            seq_name = 'seq'
            
        # store sequence subsets in a dictionary
        dic_seq = {}
        for i in range(0,3):
            a = int(i*n_segs) ; b = int(i*n_segs)+n_segs 
            identifier = f"{seq_name}_{a}:{b}"
            dic_seq[identifier] = upd_seq[a:b]
            
        lst_seq = dic_seq.values()
        index = list(dic_seq.keys())
        
        # One hot encode
        
        ii=-1
        for data in lst_seq:
            
            ii+=1; abc = 'acgt'.upper()
            
            char_to_int = dict((c, i) for i, c in enumerate(abc))
            int_enc = [char_to_int[char] for char in data]
            
            ohe = []
            for value in int_enc:
                base = [0 for _ in range(len(abc))]
                base[value] = 1
                ohe.append(base)
            np_mat = np.array(ohe)
            np_mat = np.expand_dims(np_mat,axis=0)
            
            if(ii is not 0):
                matrix = np.concatenate([np_mat,matrix],axis=0)
            else:
                matrix = np_mat
                
        return matrix,index

In [5]:
'''
#######################################################

# Attributes:
- seq - sequence in string format (passed from SQ)
- seq_type - sequence type (passed from SQ)

# Methods:
- transcription() - Change DNA to RNA
- protein(mins=0) - Decode all amino acid chains for 
                    all 6 reading frames

# Decoding of instructions for making proteins from DNA
# transcription (DNA -> RNA)
# get_protein (RNA -> AA chains containing proteins)

#######################################################
'''

class Decode(SQ):
    
    # constructor
    def __init__(self,seq,seq_type):
        self.seq = seq
        self.seq_type = seq_type
        
    @staticmethod
    def dictmap(tid=None):

        tc = {
            "GCT":"A", "GCC":"A", "GCA":"A","GCG":"A",
            "TGT":"C", "TGC":"C","GAT":"D","GAC":"D",   
            "GAA":"E", "GAG":"E","TTT":"F","TTC":"F",   
            "GGT":"G", "GGC":"G","GGA":"G","GGG":"G",
            "CAT":"H", "CAC":"H","ATA":"I","ATT":"I", "ATC":"I",  
            "AAA":"K", "AAG":"K","TTA":"L","TTG":"L", "CTT":"L",  
            "CTC":"L", "CTA":"L","CTG":"L",
            "ATG":"M", # starting codon
            "AAT":"N", "AAC":"N","CCT":"P","CCC":"P", "CCA":"P", "CCG":"P",
            "CAA":"Q", "CAG":"Q","CGT":"R","CGC":"R", "CGA":"R",
            "CGG":"R", "AGA":"R","AGG":"R","TCT":"S", "TCC":"S", "TCA":"S",
            "TCG":"S", "AGT":"S","AGC":"S","ACT":"T", "ACC":"T", "ACA":"T", 
            "ACG":"T","GTT":"V", "GTC":"V","GTA":"V","GTG":"V","TGG":"W",
            "TAT":"Y", "TAC":"Y",
            "TAA":"_","TAG":"_","TGA":"_" # ending codon
            }

        if tid in tc: 
          return tc[tid]
        else: 
          return None
        
    ''' DNA Transcription '''
    # Convert DNA -> RNA chain
    
    def transcription(self):
        if (self.seq_type == "dna"):
            return SQ(self.seq.replace("T","U"), "rna")
        else:
            print('[note] seq_type != dna')
            return None
        
    # Translate sequence
    def trans(self,seq,p0=0):
        seq_aa = ""
        for pos in range(p0,len(seq)-2,3):
            cod = seq[pos:pos+3]
            seq_aa += self.dictmap(tid=cod)
        return seq_aa
    
    '''Get All Possible open reading frames (ORF)'''
    # store all possible collections of aa
    # groups in all 6 frames
    
    def frames(self):
        res = []
        for i in range(0,3):
            res.append(self.trans(self.seq,i))
        rc = self.reverse_comp()
        for i in range(0,3):
            res.append(self.trans(rc,i)) 
        return res
    
    ''' Computes all possible proteins in an aa sequence in RF '''
    # using the knowledge that it starts with M and ends with _, 
    # filter out rule breaking ORFs
    # aa_seq -> full converted amino acid sequence
    
    @staticmethod
    def all_rf(aa_seq):
        
        current_prot = []; proteins = []
        for aa in aa_seq:
            if(aa == "_"): # stopping gap
                if(current_prot):
                    for p in current_prot:
                        proteins.append(p)
                    current_prot = []
            else: # not stopping gap
                if(aa == "M"):      # starting amino acid            
                    current_prot.append("")
                for i in range(len(current_prot)):
                    current_prot[i] += aa
                    
        return proteins
    
    ''' Computes all possible putative proteins for all ORF '''
    # and sort them based on size
    
    def protein(self,mins=0):
        
        # order 
        def insert_prot_ord(prot, lst_prot):
            i = 0
            while(i < len(lst_prot) and len(prot)<len(lst_prot[i])):
                i += 1
            lst_prot.insert(i, prot)
            
        # get all ORF conversions
        rfs = self.frames();res = []
        for rf in rfs:
            # return only protein cases
            prots = self.all_rf(rf) 
            # additionally sort based on protein size
            for p in prots: 
                if(len(p) > mins): 
                    insert_prot_ord(p,res)
        return res


In [6]:
'''

#####################################################

# Counting
# freq - count bases in sequence
# count_purines - count purines & pyrimidines
# groupfreq - count grouped bases

#####################################################

'''

class Count:
    
    def __init__(self,seq,seq_type,id):
        self.seq = seq
        self.seq_type = seq_type
        self.id = id
        
    @staticmethod
    def dict_sum(dictlist):
        outdic = {}
        for d in dictlist:
            for k in d.keys():
                outdic[k] = 0
        for d in dictlist:
            for k in d.keys():
                outdic[k]+=d[k]
        return outdic
        
    # Get ABC
    def abc(self):
        if(self.seq_type=="dna"): 
          return "ACGT"
        elif(self.seq_type=="rna"):
          return "ACGU"
        elif (self.seq_type=="aa"): 
          return "ACDEFGHIKLMNPQRSTVWY"
        else: 
          return None
    
    # Mapping Dictionary
    @staticmethod
    def dictmap(map_id='iupac_amino',tid=None):

        # IUPAC Amino Acids
        if(map_id is 'iupac_amino'):
            tc   = {'A':'Alanine','C':'Cysteine','D':'Aspartic Acid',
                    'E':'Glutamic Acid','F':'Phenylalanine','G':'Glycine',
                    'H':'Histidine','I':'Isoleucine','L':'Lysine',
                    'M':'Methionine','N':'Asparagine','P':'Proline',
                    'Q':'Glutamine','R':'Arginine','S':'Serine',
                    'T':'Threonine','V':'Valine','W':'Tryptophan',
                    'Y':'Tryosine','_':'Gap'}

        # IUPAC nuceotides
        elif(map_id is 'iupac_nucleotide'):
            tc  = {'A':'Adenine','C':'Cytosine','G':'Guanine','T':'Thymine',
                   'U':'Uracil'}

        if tid in tc: 
          return tc[tid]
        else: 
          return None
        
    ''' Frequency of Alphabet '''
    # nucleotide or amino acid sequences
    
    def freq(self,compare:list=None,       # list of comparing sequences
                  show_id:str='perc',     # perc/count
                  fsize:list=[None,None],  # figure size
                  title:str=None,
                  barmode:str='group'): 
        
        c1 = dict(Counter(self.seq))  # abc counter for s1
        abc = list(self.abc())
        count = Counter(abc)
        abc_c = dict(Counter({x:0 for x in count}))
        c_all1 = self.dict_sum([c1,abc_c])
            
        lst = []
        for i in c_all1.keys():
           if(self.seq_type == 'dna' or self.seq_type == 'rna'):
               lst.append(self.dictmap('iupac_nucleotide',i))
           elif(self.seq_type == 'aa'):
               lst.append(self.dictmap('iupac_amino',i))
                    
        perc = [round(x/len(self.seq),3)*100 for x in [*c_all1.values()]]
        if(show_id is 'perc'):
            show1 = lst; show2 = perc;
            xaxis_id = 'Character (%)'
        elif(show_id is 'count'):
            show1 = lst; show2 = [*c_all1.values()]
            xaxis_id = 'Count'
            
        if(self.id is not None):
            lname = self.id
        else:
            lname = 'main seq'
            
        fig = go.Figure(go.Bar(y=show1,x=show2,
                               orientation='h',name=lname))
        
        # Compare other sequences
        if(compare is not None):
            
            # Cycle through all SQ in list
            
            for ii,lseq in enumerate(compare):
                
                ii+=1;
                c2 = dict(Counter(lseq.seq))  # abc counter for s2
                c_all2 = self.dict_sum([c2,abc_c])    
            
                lst2 = []
                for i in c_all2.keys():
                   if(self.seq_type == 'dna' or self.seq_type == 'rna'):
                       lst2.append(self.dictmap('iupac_nucleotide',i))
                   elif(self.seq_type == 'aa'):
                       lst2.append(self.dictmap('iupac_amino',i))
                
                perc = [round(x/len(lseq.seq),3)*100 for x in [*c_all2.values()]]
                if(show_id is 'perc'):
                    show1 = lst2;show2 = perc
                elif(show_id is 'count'):
                    show1 = lst2; show2 = [*c_all2.values()]
                
                if(lseq.id is not None):
                    lname = lseq.id
                else:
                    lname = f'seq{ii}'
                    
                fig.add_trace(go.Bar(y=show1,x=show2,
                                     orientation='h',
                                     name=lname))
                
        # Set title
        
        if(title is None):
            title = f'{self.seq_type} sequence content'
        else:
            title = title
            
        fig.update_layout(template='plotly_white',
                          barmode=barmode, # stack,group,overlay,relative
                          height=fsize[0],width=fsize[1],
                          bargroupgap=0.2, 
                          font=dict(size=11),title=title)
        fig.update_traces(width=0.25,
#                           marker_color='rgb(158,202,225)', 
                          marker_line_color='#212121',
                          marker_line_width=1.0, opacity=0.6)
        fig.update_xaxes(nticks=20,title_text=xaxis_id);fig.show()
        
    # Count frequency of grouped nucleotides
    
    def gfreq(self,compare:list=None,
                   count_id='di',
                   fsize=[None,None],
                   barmode='group',
                   xlim=20):
        
        if(count_id is 'di'):
            lst_count_id = ['AA','AC','AG','AT',
                            'CA','CC','CG','CT',
                            'GA','GC','GG','GT',
                            'TA','TC','TG','TT']
            
        elif(count_id is 'tri'):
            lst_count_id = ['AAA','AAC','AAG','AAT','ACA','ACC','ACG',
                            'ACT','AGA','AGC','AGG','AGT','ATA','ATC',
                            'ATG','ATT','CAA','CAC','CAG','CAT','CCA',
                            'CCC','CCG','CCT','CGA','CGC','CGG','CGT',
                            'CTA','CTC','CTG','CTT','GAA','GAC','GAG',
                            'GAT','GCA','GCC','GCG','GCT','GGA','GGC',
                            'GGG','GGT','GTA','GTC','GTG','GTT','TAA',
                            'TAC','TAG','TAT','TCA','TCC','TCG','TCT',
                            'TGA','TGC','TGG','TGT','TTA','TTC','TTG',
                            'TTT']
            
        if(self.seq_type is 'dna'):
            
            lst_c = []
            for i in lst_count_id:
                lst_c.append(self.seq.count(i))
                
            df = pd.DataFrame(data=lst_c,
                              index=lst_count_id).T
            df.index = [self.id]
            
            if(compare is not None):
                
                ii=-1
                for seq in compare: # cycle through all SQ
                
                    ii+=1;lst_c = []
                    for jj in lst_count_id:
                        lst_c.append(compare[ii].seq.count(jj))
                        
                    ldf = pd.DataFrame(data=lst_c,
                                       index=lst_count_id).T
                    ldf.index = [seq.id]
                    df = pd.concat([df,ldf],axis=0)
            
            if(count_id is 'tri'):
                xaxis=dict(rangeslider=dict(visible=True))
            else:
                xaxis = None
                
            # Plot
            fig = px.bar(df.T,
                         x=df.T.index,
                         y=df.T.columns)
            fig.update_layout(template='plotly_white',
                              barmode=barmode,
                              title=f'{count_id} nucleotide | count',
                              font={'size':11},
#                               margin=dict(l=20, r=20, t=80, b=20),
                              xaxis=xaxis,
                              height=fsize[0],width=fsize[1]); 
            fig.update_traces(width=0.25,
    #                           marker_color='rgb(158,202,225)', 
                              marker_line_color='#212121',
                              marker_line_width=1.0, opacity=0.6)
            
            if(count_id is 'tri'):
                fig['layout']['xaxis'].update(title='', 
                                              range=[-1,xlim], 
                                              autorange=False) # range slider
            
            fig.show()
        else:
            print('[note] input must be dna type')
            
    # Return purines & pyrimidines (%)
    def purines(self):
        
        if (self.seq_type == "dna" or self.seq_type == "rna"):        
        
            dic_count = {}
            purines1 = self.seq.count("A") + self.seq.count("G")
            pyrimidines1 = self.seq.count("C") + self.seq.count("T")
            purines1 = purines1/len(self.seq)
            pyrimidines1 = pyrimidines1/len(self.seq)
            dic_count['purines'] = round(purines1,4)
            dic_count['pyrimidine'] = round(pyrimidines1,4)
            return dic_count
    
    # Return GC nucleotide (%)
    def gc(self):
        
        if (self.seq_type == "dna" or self.seq_type == "rna"):
            ii = 0
            for s in self.seq:
                if(s in "GC"):
                    ii += 1

            val = round(ii/len(self.seq),4)
            return val

In [7]:
''' 

#####################################################
    
# Finding Patterns in Sequence
# find - find index(ies) of particular pattern 
    
#####################################################

'''

class Pattern:
    
    def __init__(self,seq,seq_type):
        self.seq = seq
        self.seq_type = seq_type
    
    # Prosite Pattern Lines
    # - Standard IUPAC amino acid used to as bases in pattern, separated by -
    # - x -> any amino acid acceptable
    # - [] -> ambiguity represented by list, any aa in that list acceptable
    # - {} -> ambiguity represented by list, any aa other than in {} accepted
    # - repetition of pattern element shown below:
    #   x(3) -> x-x-x, x(2,4) -> to x-x or x-x-x or x-x-x-x
    
    @staticmethod
    def prosite_process(rex):
        # adjust prosite to RE format
        rex = rex.replace("(","{")
        rex = rex.replace(")","}")
        rex = rex.replace("x",".")
        rex = rex.replace("-","")
        return rex
    
    ''' Find Substring Pattern in Sequence '''
    
    def find(self,pattern,          # subsequence pattern
                  find_id='first',  # first,all,overlap
                  search_id=None,   # search pattern option (prosite)
                  verbose=True):    # Verbal Output
        
        ''' [1] Find First Match Only '''
        
        if(find_id is 'first'):
            
            if(search_id is 'prosite'):
                pattern = self.prosite_process(pattern)
                
            # General search as well
            re_search = search(pattern,self.seq)
            if (re_search != None):
                if(verbose):
                    print(f"showing first for {pattern}")
                result = re_search.span()[0]
                return result
            else:
                if(verbose):
                    print(f'no matches for {pattern} found')
                
        elif(find_id == 'all'):
            
            if(search_id is 'prosite'):
                pattern = self.prosite_process(pattern)
                
            re_search = finditer(pattern,self.seq)
            result = []
            for x in re_search:
                result.append(x.span()[0])
                
            if(len(result) is not 0):
                if(verbose):
                    print(f"found {len(result)} matches")
                return result
            else:
                if(verbose):
                    print(f'no matches for {pattern} found')
                
        elif(find_id == 'overlap'):
            
            if(search_id is 'prosite'):
                pattern = self.prosite_process(pattern)
            mos = finditer("(?="+pattern+")",self.seq)
            result = []
            for x in mos:
                result.append(x.span()[0])
                
            if(len(result) is not 0):
                if(verbose):
                    print(f"found {len(result)} matches")
                return result
            else:
                if(verbose):
                    print(f'no matches for {pattern} found')
        else:
            print('first,all,overlap options')

# <div style="padding: 30px;color:white;margin:10;font-size:80%;text-align:left;display:fill;border-radius:10px;background-color:#F1C40F;overflow:hidden;background-image: url(https://i.imgur.com/tNNAhbu.png)"><b><span style='color:white'>3 |</span></b> Basic Sequence Operations</div>

<div style="color:white;display:fill;border-radius:8px;
            background-color:#03112A;font-size:150%;
            letter-spacing:1.0px">
    <p style="padding: 8px;color:white;"><b><b><span style='color:#F1A424'>3.1 |</span></b> Section Overview</b></p>
</div>

- In this section, we'll look at **basic operations** associated with our classes, focusing on just using the **availble commands** assoiated with blocks

> - **<mark style="background-color:#212121;color:white;border-radius:5px;opacity:0.8">3.2</mark>** - Define a **sequence** from a python string
> - **<mark style="background-color:#212121;color:white;border-radius:5px;opacity:0.8">3.3</mark>** - A string must contain characters from either the nucleotide or animo acid alphabet, let's check if our sequence has any missprints
> - **<mark style="background-color:#212121;color:white;border-radius:5px;opacity:0.8">3.4</mark>** - All DNA strings come in pairs, we always input one of its strands, let's obtain the **reverse** of the complementary strand
> - **<mark style="background-color:#212121;color:white;border-radius:5px;opacity:0.8">3.5</mark>** - sequences in SQ only need to be initialised with the sequence itself, however we can also add mode information about out sequence
> - **<mark style="background-color:#212121;color:white;border-radius:5px;opacity:0.8">3.6</mark>** - Each sequence may contain subsequences or regions of interest, we can add annotations that will specify the region location & a description about this subsequence

<br>

<div style="color:white;display:fill;border-radius:8px;
            background-color:#03112A;font-size:150%;
            letter-spacing:1.0px">
    <p style="padding: 8px;color:white;"><b><b><span style='color:#F1A424'>3.2 |</span></b> Defining Basic Sequences from Strings</b></p>
</div>

- We can create an instance of object **<mark style="background-color:#212121;color:white;border-radius:5px;opacity:0.8">SQ</mark>** using the following ( including the <b>sequence type</b> )
- The **sequence** and **sequence type** summary can be view by calling **<mark style="background-color:#212121;color:white;border-radius:5px;opacity:0.8">info</mark>** method
- DNA sequences are also colour coded, making it a little easier to distinguish the different characters
- We can also view the sequence by calling the **<mark style="background-color:#212121;color:white;border-radius:5px;opacity:0.8">view</mark>** method, which accepts an argument <code>split_id</code> (indicating after how many characters a pause should be made when visualising it)

In [8]:
# Define sequences in string format
seqn = 'ATGACGGATCAGCCGCAAGCGGAATTGGCGTTTACGTACGATGCGCCGTAA'
seqaa = 'MMMELQHQRLMALAGQLQLESLISAAPALSQQAVDQEWSYMDFLEHLLHDFTF'

# Define Two DNA sequences (instances)
sq_n = SQ(seqn,'dna')
# define new protein sequence 
sq_aa = SQ(seqaa,'aa')

# Show class variables
sq_n.info()
sq_aa.info()

SEQ: ATGACGGATCAGCCGCAAGCGGAATTGGCGTTTACGTACGATGCGCCGTAA TYPE: dna
SEQ: MMMELQHQRLMALAGQLQLESLISAAPALSQQAVDQEWSYMDFLEHLLHDFTF TYPE: aa


In [9]:
# Show DNA sequence using 5 character batches
sq_n.view(5)

ATGAC GGATC AGCCG CAAGC GGAAT TGGCG
TTTAC GTACG ATGCG CCGTA A

<div style="color:white;display:fill;border-radius:8px;
            background-color:#03112A;font-size:150%;
            letter-spacing:1.0px">
    <p style="padding: 8px;color:white;"><b><b><span style='color:#F1A424'>3.3 |</span></b> Check Validity of Strings</b></p>
</div>

- A sequence is valid if its alphabet corresponds to a specific set of code, class **<mark style="background-color:#212121;color:white;border-radius:5px;opacity:0.8">SQ</mark>** uses the **[IUPAC](https://www.bioinformatics.org/sms2/iupac.html)** code standard
- We can check whether a sequence is valid or not by using the **<mark style="background-color:#212121;color:white;border-radius:5px;opacity:0.8">validate</mark>** function, returning a logical output
- This can be handy when our sequece is very large, and we can check the entire sequence for errors

In [10]:
print(f'sequence: {sq_aa.seq} is valid: {sq_aa.validate()}')

sequence: MMMELQHQRLMALAGQLQLESLISAAPALSQQAVDQEWSYMDFLEHLLHDFTF is valid: True


<div style="color:white;display:fill;border-radius:8px;
            background-color:#03112A;font-size:150%;
            letter-spacing:1.0px">
    <p style="padding: 8px;color:white;"><b><b><span style='color:#F1A424'>3.4 |</span></b> Switch to its Reverse Complement</b></p>
</div>

- DNA has **<span style='color:#F1C40F'>two complementary strands</span>**
- Due to the complementarity of the DNA strands, usually only one of the strands is provided in a sequence files obtained from databases
- The second strand to the input <b>DNA sequence</b> can be obtained by calling the **<mark style="background-color:#212121;color:white;border-radius:5px;opacity:0.8">reverse_comp</mark>** function

In [11]:
rev_comp = sq_n.reverse_comp()
rev_comp.info()

SEQ: TTACGGCGCATCGTACGTAAACGCCAATTCCGCTTGCGGCTGATCCGTCAT TYPE: dna


<div style="color:white;display:fill;border-radius:8px;
            background-color:#03112A;font-size:150%;
            letter-spacing:1.0px">
    <p style="padding: 8px;color:white;"><b><b><span style='color:#F1A424'>3.5 |</span></b> Defining a Detailed Sequence</b></p>
</div>

- **<mark style="background-color:#212121;color:white;border-radius:5px;opacity:0.8">SQ</mark>** can also be used to store more detailed information about our sequence; <code>id</code> and <code>description</code>
- Later we'll need to look at a sequence & jot down some information about parts of the sequence after we have identified them as well

In [12]:
# Defining some extra detail
seq_det = SQ(seq='ACTTTTGACCTCAA',     # sequence
            id='sequence',             # give the sequence some identifier 
            description='mysequence')  # write a short description about the sequence

# When required, we can call the SQ object by using .seq
print(f'Extracting SQ when needed: {seq_det.seq}')

Extracting SQ when needed: ACTTTTGACCTCAA


In [13]:
print(f'Sequence ID: {seq_det.id}') # used for alignment naming etc
print(f'Sequence Annotations for subsequence labeling: {seq_det.annotations}')
print(f'A note describing the sequence: {seq_det.description}')

Sequence ID: sequence
Sequence Annotations for subsequence labeling: {}
A note describing the sequence: mysequence


<div style="color:white;display:fill;border-radius:8px;
            background-color:#03112A;font-size:150%;
            letter-spacing:1.0px">
    <p style="padding: 8px;color:white;"><b><b><span style='color:#F1A424'>3.6 |</span></b> Making Annotations</b></p>
</div>

- Say we searched a database to find if our sequence contains something interesting
- We have two hits, **'TTTT'**,**'CCTCA'**, which we want to save in <code>annotation</code>
- Usually, when we search databases, we'll have a **sequence identifier** & **a description/comment** about the sequence, so we'll be able to save this info
- The search result sequences can be extracted from the **alignment format** & saved in **<mark style="background-color:#212121;color:white;border-radius:5px;opacity:0.8">SQ</mark>** format

In [14]:
subsequence1 = SQ('TTTT',id='db|1',description='subseq') 
subsequence2 = SQ('CCTCA',id='db|2',description='subseq')

- Next, we can find where it is located in the sequence, using yet to be introduced method; **<mark style="background-color:#212121;color:white;border-radius:5px;opacity:0.8">find_pattern</mark>**
- This will be handy in **Section 8**, where we'll be searching & annotating a sequence of interest

In [15]:
seq_det.add_annotation(subsequence1)
seq_det.add_annotation(subsequence2)

In [16]:
seq_det.annotations

{'sq_2:6': 'db|1_subseq', 'sq_8:13': 'db|2_subseq'}


# <div style="padding: 30px;color:white;margin:10;font-size:80%;text-align:left;display:fill;border-radius:10px;background-color:#F1C40F;overflow:hidden;background-image: url(https://i.imgur.com/tNNAhbu.png)"><b><span style='color:white'>4 |</span></b> Counting Alphabet Characters</div>

<div style="color:white;display:fill;border-radius:8px;
            background-color:#03112A;font-size:150%;
            letter-spacing:1.0px">
    <p style="padding: 8px;color:white;"><b><b><span style='color:#F1A424'>4.1 |</span></b> Nucleotides % in a Sequence</b></p>
</div>

- We may want to obtain the count for each each **nucleotide** or **amino acid** in the given sequence that has the same amount of bases
- We can use the **<mark style="background-color:#212121;color:white;border-radius:5px;opacity:0.8">freq</mark>** method to show:
> - The **count**/**percentage** of each of the alphabet **compare two sets** of sequences with the addition of <b>compare</b>

In [17]:
# compare content percentage (default)

sq_n_compare = SQ('ACTCCCTGAGGATCATGC',id='sequence A') # sequence with identifier shows up in graph
sq_n_compare2 = SQ('ACATACATAACACATCC')                 # sequences w/o id are set to default names

# Compare the nucleotide % of three sequences

sq_n.count.freq(compare=[sq_n_compare,sq_n_compare2], # list of sequences
                fsize=[400,800],
                show_id='count',
                title='Comparing Nucleotide Content (%)')

<div style="color:white;display:fill;border-radius:8px;
            background-color:#03112A;font-size:150%;
            letter-spacing:1.0px">
    <p style="padding: 8px;color:white;"><b><b><span style='color:#F1A424'>4.2 |</span></b> Counting Amino Acids in a Sequence</b></p>
</div>

- We may want to compare sequences of **<span style='color:#F1C40F'>different length</span>**, so in that case it's best to use **percentage (default)**
- <code>show_id='count'</code> is good for when we just want to see the character count of sequence

In [18]:
# show the amino acid content of the protein

sq_aa.count.freq(show_id='perc', 
                 fsize=[500,800])

<div style="color:white;display:fill;border-radius:8px;
            background-color:#03112A;font-size:150%;
            letter-spacing:1.0px">
    <p style="padding: 8px;color:white;"><b><b><span style='color:#F1A424'>4.3 |</span></b> Di/Trinucleotides</b></p>
</div>

- We may want to know the different combinations of <b><span style='color:#F1C40F'>double</span></b> or <b><span style='color:#F1C40F'>tripple nucleotides</span></b> that is present within the sequence
- One example would be to count all the <b><span style='color:#F1C40F'>codons</span></b> present in a sequence

In [19]:
# Define Two DNA sequences (instances)
sq_1 = SQ('ATGACGGATCAGCCGCAAGCGGAATTGGCGCACATTTACGTACGATGCGCCGTAA','dna',id='seq1')
sq_2 = SQ('ATGACGGATCAGCCGCAAGCGGAATTGGCGTTTACGTACGAAGAGACACCCGTAA','dna',id='seq2')
sq_3 = SQ('ATTTAATCATTGCCGTTATCGGAATTGGCGTTTACGTACGAAGAGACACCCGTAA','dna',id='seq3')

# Compare Trinucleotides of two sequences
sq_1.count.gfreq(compare=[sq_2,sq_3],
                 count_id='tri',
                 barmode = 'group', # group,stack
                 fsize=[300,800])

In [20]:
## Compare Dinucleotides of two sequences
sq_n.count.gfreq(compare=[sq_1,sq_2,sq_3],
                   count_id='di',
                   barmode = 'stack', # group,stack
                   fsize=[400,800])

<div style="color:white;display:fill;border-radius:8px;
            background-color:#03112A;font-size:150%;
            letter-spacing:1.0px">
    <p style="padding: 8px;color:white;"><b><b><span style='color:#F1A424'>4.4 |</span></b> GC Content</b></p>
</div>

**<mark style="background-color:#F1C40F;color:white;border-radius:5px;opacity:0.9">GC-Content</mark>** | **[WIKIPEDIA](https://en.wikipedia.org/wiki/GC-content)** & **[SCIENCEDIRECT](https://www.sciencedirect.com/topics/biochemistry-genetics-and-molecular-biology/gc-content)**

> - In molecular biology and genetics, GC-content (or guanine-cytosine content) is the percentage of nitrogenous bases in a DNA or RNA molecule that are either <b>guanine</b> (G) or <b>cytosine</b> (C). This measure indicates the proportion of G and C bases out of an implied four total bases, also including adenine and thymine in DNA and adenine and uracil in RNA.
> - GC content is strongly correlated with biological features of genome organization such as distribution of various classes of repeated elements, gene density, level and tissue-specificity of transcription, and mutation rate.

In [21]:
print(f'GC content of ({sq_n.name}) : {round(sq_n.count.gc(),3)}')

GC content of (name not given) : 0.549


<div style="color:white;display:fill;border-radius:8px;
            background-color:#03112A;font-size:150%;
            letter-spacing:1.0px">
    <p style="padding: 8px;color:white;"><b><b><span style='color:#F1A424'>4.5 |</span></b> Purines & Pyrimidines</b></p>
</div>

- **<mark style="background-color:#F1C40F;color:white;border-radius:5px;opacity:0.9">Purines</mark>** - (adenine and guanine) have a two-ringed structure consisting of a nine-membered molecule with four nitrogen atoms
- **<mark style="background-color:#F1C40F;color:white;border-radius:5px;opacity:0.9">Pyrimidines</mark>** - (cytosine, uracil, and thymine) only have one single ring, which has just six members and two nitrogen atoms

In [22]:
print('dict w/ percentages:')
sq_n.count.purines()

dict w/ percentages:


{'purines': 0.5686, 'pyrimidine': 0.4314}

# <div style="padding: 30px;color:white;margin:10;font-size:80%;text-align:left;display:fill;border-radius:10px;background-color:#F1C40F;overflow:hidden;background-image: url(https://i.imgur.com/tNNAhbu.png)"><b><span style='color:white'>5 |</span></b> Decoding DNA for Protein Generation</div>

<div style="color:white;display:fill;border-radius:8px;
            background-color:#03112A;font-size:150%;
            letter-spacing:1.0px">
    <p style="padding: 8px;color:white;"><b><b><span style='color:#F1A424'>5.1 |</span></b> Transcription</b></p>
</div>

- The mRNA is created as the two complementary strands are split, and is complement to one of the strands.
- Transcription of <b>DNA sequence</b> can be obtained using the <code>.transcription()</code> function, where the <b>T</b> is replaced by <b>U</b> in the sequence string.

In [23]:
rna_seq = sq_n.decode.transcription()
rna_seq.info()

SEQ: AUGACGGAUCAGCCGCAAGCGGAAUUGGCGUUUACGUACGAUGCGCCGUAA TYPE: rna


<div style="color:white;display:fill;border-radius:8px;
            background-color:#03112A;font-size:150%;
            letter-spacing:1.0px">
    <p style="padding: 8px;color:white;"><b><b><span style='color:#F1A424'>5.2 |</span></b> Translation</b></p>
</div>

- The background to what happens during the translation process was already outlined in 1.5, however it makes sense to expand on theory a little more.
- Proteins are synthesised by creating <b>chains of aminoacids</b>, according to information contained in the <b>messenger RNA (mRNA)</b> in a process called <b>translation</b>.

#### <b><span style='color:#F1C40F'>START & STOP CODONS</span></b>
- **<mark style="background-color:#F1C40F;color:white;border-radius:5px;opacity:0.9">Translation</mark>** of a protein:
    - Always begins with a specific codon <b>ATG</b> -> <b>M (Methionine)</b>, which is always the first amino in the protein.
- <b>Translation</b> process terminates when a stop codon is found; <b>TAA</b>,<b>TAG</b>,<b>TGA</b> -> <code>_</code>.
- An example sequence where we know exactly where the <b>start</b> and <b>termination codons</b> are:
    - **ATG**<code>ACGGATCAGCCGCAAGCGGAATTGGCGTTTACGTACGATGCGCCG</code>**TAA**
    - We can note it **starts with ATG** & ends with **one of the three stop codons**.
- As the sequence follows the rule of start and ending codon, in such a case we can use the <b>staticmethod</b> defined in SQ; <code>.translate(p0=0)</code> directly.
- Some interesting discussions about the **[starting amino acid](https://www.researchgate.net/post/Does_every_protein_start_with_methionine)** (ATG is usually the starting codon)

#### <b><span style='color:#F1C40F'>OPEN READING FRAME</span></b>
- A <b>reading frame</b> is a way of dividing the DNA sequence into a set of consecutive, <b>non-overlapping triplet nucleotides</b> (possible codons) (using dictionary mapping).
- A given sequence has 3 possible reading frames, first, second and third nucleotide positions. In addition, considering there is another complementary strand, we should compute the only 3 frames corresponding to the reverse compliment.
- In many cases, given a DNA sequence, <b>we don't know in advance where the coding regions are</b>, especially when dealing with complete sequences.
- In such cases, we need to <b>scan the DNA sequence for the coding region</b>. First, we need to divide and compute these reading frames (6 in total). The <code>frames</code> function stores the converted 6 converted amino acid strings in a list.

<b>OPEN READING FRAME</b> | [genomove.gov](https://www.genome.gov/genetics-glossary/Open-Reading-Frame)

> - An open reading frame is a portion of a DNA molecule that, when translated into amino acids, contains no stop codons
> - The genetic code reads DNA sequences in groups of three base pairs, which means that a double-stranded DNA molecule can read in any of six possible reading frames: three in the forward direction and three in the reverse.
> - A long open reading frame is likely part of a gene.
    
| Open Reading Frames (ORF)[[1]](https://www.genome.gov/genetics-glossary/Open-Reading-Frame) |
| - |
|<img src="https://www.genome.gov/sites/default/files/tg/en/illustration/open_reading_frame.jpg" alt="Drawing" style="width:700px;"/> |

#### <b><span style='color:#F1C40F'>IDEAL CASE (P=0)</span></b>
- Let's try one of the reading frames at the start of the sequence (<b><span style='color:#5D2ECC'>p0=0</span></b>), which we know follows the correct rules observed in life.
- Let's also consider all other possible ORFS & get the final protein found in the DNA sequence.

In [24]:
''' Don't actually have to call (just for demonstration) '''

# Sequence with initial codon coinciding with start codon &
# end w/ end codon (ATG & TAA respectively)
seq = 'ATGACGGATCAGCCGCAAGCGGAATTGGCGTTTACGTACGATGCGCCGTAA'
strand = SQ(seq=seq,seq_type='dna')
print(f'Correct ORF: {strand.decode.trans(strand.seq,p0=0)}')

print(f'All ORF:')
lst_ORF = strand.decode.frames()
lstcol(lst_ORF)

Correct ORF: MTDQPQAELAFTYDAP_
All ORF:
MTDQPQAELAFTYDAP_    DGSAASGIGVYVRCAV     YGASYVNANSACG_SV     
_RISRKRNWRLRTMRR     LRRIVRKRQFRLRLIRH    TAHRT_TPIPLAADPS     


In [25]:
# Only one of the ORFs meets the requirement, so only one putative protein is found
proteins = strand.decode.protein()
print(proteins)
print(f'{len(proteins[0])} characters')

['MTDQPQAELAFTYDAP']
16 characters


Some interesting information about how many amino acids do we need to actually classify the decoded result as a protein
- **[proteins & amino acids](https://www.ncbi.nlm.nih.gov/books/NBK234922/)** (Both animal and plant proteins are made up of about 20 common amino acids)
- We will mention those with **less than about 20 amino acids** below when we'll look at an example

# <div style="padding: 30px;color:white;margin:10;font-size:80%;text-align:left;display:fill;border-radius:10px;background-color:#F1C40F;overflow:hidden;background-image: url(https://i.imgur.com/tNNAhbu.png)"><b><span style='color:white'>6 |</span></b> Loading Files Containing Sequences</div>

#### <b><span style='color:#F1C40F'>REAL SEQUENCES</span></b>

- Most realistic application of bioinformatics certainly involve working with <b>sequences that are too long</b>
- Often we need to work with databases, so it's convenient to have an effective I/O system in place
- One of such formats is <code>FASTA</code>, its made to be a very flexible format, allowing us to work with single, multiple or even alignment sequences.

#### <b><span style='color:#F1C40F'>THE FASTA FORMAT</span></b>

**<mark style="background-color:#F1C40F;color:white;border-radius:5px;opacity:0.9">FASTA</mark>** | **[FASTA format](https://en.wikipedia.org/wiki/FASTA_format)**:
>In bioinformatics and biochemistry, the FASTA format is a text-based format for representing either <b>nucleotide sequences</b> or <b>amino acid (protein) sequences</b>, in which nucleotides or amino acids are represented using single-letter codes. The format also allows for sequence names and comments to precede the sequences. The format originates from the FASTA software package, but has now become a near universal standard in the field of bioinformatics.

- Commonly used to store nucleotide or protein sequences. It's less detailed & usually containing only the <b>sequence</b>, and <b>name/header</b> only.
- <b><span style='color:#F1C40F'>File extension</span></b> | Typically changed based on the <b>sequence type</b> content of the file, defined below in the dictionary <code>FASTA_dic</code>.
- <b><span style='color:#F1C40F'>Multisequences</span></b> | The format can contain any number of sequences, each starting with the symbol: <b>></b>.
- The <b><span style='color:#F1C40F'>name/header</span></b>, defined after the symbol <b>></b> contains an origin identifier (<b><span style='color:#F1C40F'>NCBI identifiers</span></b>), defined in the the dictionary <code>identifiers_dic</code>.

> - The NCBI defined a standard for the unique identifier used for the sequence (SeqID) in the header line. 
> - This allows a sequence that was obtained from a database to be labelled with a reference to its database record. 
> - The database identifier format is understood by the NCBI tools like makeblastdb and table2asn. 
> - The following list describes the NCBI FASTA defined format for sequence identifiers

<div style="color:white;display:fill;border-radius:8px;
            background-color:#03112A;font-size:150%;
            letter-spacing:1.0px">
    <p style="padding: 8px;color:white;"><b><b><span style='color:#F1A424'>CLASS |</span></b> read_seq</b></p>
</div>


#### <b><span style='color:#F1C40F'>READING SEQUENCE FILES CLASS</span></b>

- We define a custom class, <code>read_seq()</code>, which methods used for reading the <code>FASTA</code> format
- The class automatically should detect the class type and call the corresponding class, storing the sequence upon instantiation
- Sequences that are read are stored in <code>SQ</code> objects, which allows us to also to also store additional information about the sequences

**<mark style="background-color:#212121;color:white;border-radius:5px;opacity:0.8">CONSTRUCTOR</mark>**
- <code>name</code> : stores the **<span style='color:#F1C40F'>pathway</span>** to the file
- <code>format</code> : stores the **<span style='color:#F1C40F'>FASTA format</span>** identifier

**<mark style="background-color:#212121;color:white;border-radius:5px;opacity:0.8">CLASS METHODS</mark>**
- <code>read_FASTA</code> : used to read <code>FASTA</code> formats, storing sequence data in <code>lst_seq</code> & descriptions in <code>SQ</code> objects
- <code>get_sq</code> : used to store the return a list or a sequence of <code>SQ</code> class objects

In [26]:
# NCBI identifiers
identifiers_dic = {'lcl':'local(nodb)','bbs':'GenInfo backbone seqid',
                   'bbm':'GenInfo backbone moltype','gim':'GenInfo import ID',
                   'gb':'GenBank','emb':'EMBL','pir':'PIR','sp':'SWISS-PROT',
                   'pat':'patent','pgp':'pre-grant patent','ref':'RefSeq',
                   'gnl':'general database reference','prf':'PRF','pdb':'PDB',
                   'gi':'GenInfo integrated database','dbj':'DDBJ'}

# FASTA formats
FASTA_dic = {'fa':'generic','fasta':'generic','fna':'nucleic acid',
             'ffn':'nucleotide of gene regions','faa':'amino acid',
             'frn':'non-coding RNA'}

# Class to read different files and store info only
class read_seq(SQ):
    
    def __init__(self,name):
        self.name = name
        self.format = name.rsplit('.',1)[1]    
        if(self.format in FASTA_dic):      # if one of the fasta formats
            self.read_FASTA(self.name)

    # read FASTA format
    def read_FASTA(self,filename):

        tseq = None; self.lst_seq = []     # list of sequences
        thead = None; self.lst_header = [] # list of sequence identifications
        ff = FASTA_dic[filename.rsplit('.',1)[1]]
        file = open(filename,'r')

        for line in file:
            if(search(">.*", line)): # get lines w/ >
                    if(tseq != None and thead != None and tseq != ""):
                        self.lst_seq.append(tseq)
                    thead = line; self.lst_header.append(line)              
                    tseq = ""
            else:
                if(tseq == None):
                    return None
                else: 
                    tseq += sub("\s","",line)

        if(tseq != None and thead != None and tseq != ""):
            self.lst_seq.append(tseq)
            
        print(f'READ -> FASTA [{ff}] | #SEQ: {len(self.lst_seq)}')
        file.close()
        
    # get read sequences
    def get_sq(self):
        lst_out = []
        
        # If there's more than one sequence
        if(len(self.lst_seq) > 1):
            for i in range(0,len(self.lst_seq)):
                lst_types = ['dna','rna','aa']
                for check in lst_types:
                    if(SQ(seq=self.lst_seq[i],seq_type=check).validate()):
                        lst_out.append(SQ(self.lst_seq[i],seq_type=check,
                                          description=self.lst_header[i]))
            return lst_out
        
        # return just the one file
        else:
            lst_types = ['dna','rna','aa']
            for check in lst_types:
                if(SQ(self.lst_seq[0],check).validate()): # if valid sq
                    return SQ(seq=self.lst_seq[0],seq_type=check,
                              description=self.lst_header[0])

Two examples are shown below, reading files: 
> <code>file_faa</code> & <code>file_fna</code>, containing a <b>set of protein sequences (faa)</b> and a <b>single nucleotide sequence (fna)</b> resepectively

In [27]:
# define pathway to FASTA file
file_faa = '/NC_005816.faa' # amino acid chain sequence
file_fna = '/NC_005816.fna' # nucleotide sequence 

# fetch sequence from file and store each in sequence class, SQ
col_seq_aa = read_seq(file_faa).get_sq()
col_seq_n = read_seq(file_fna).get_sq()

READ -> FASTA [amino acid] | #SEQ: 10
READ -> FASTA [nucleic acid] | #SEQ: 1


In [28]:
# sequence description
col_seq_aa[0].description

'>gi|45478712|ref|NP_995567.1| putative transposase [Yersinia pestis biovar Microtus str. 91001]\n'

In [29]:
# show sequence information; seq and type
col_seq_aa[1].info()

SEQ: MMMELQHQRLMALAGQLQLESLISAAPALSQQAVDQEWSYMDFLEHLLHEEKLARHQRKQAMYTRMAAFPAVKTFEEYDFTFATGAPQKQLQSLRSLSFIERNENIVLLGPSGVGKTHLAIAMGYEAVRAGIKVRFTTAADLLLQLSTAQRQGRYKTTLQRGVMAPRLLIIDEIGYLPFSQEEAKLFFQVIAKRYEKSAMILTSNLPFGQWDQTFAGDAALTSAMLDRILHHSHVVQIKGESYRLRQKRKAGVIAEANPE TYPE: aa


In [30]:
print(f'Number of Sequences stored: {len(col_seq_aa)}'); 
print(f'List of Sequences Type: {type(col_seq_aa)}')
print(f'List Content Type: {type(col_seq_aa[0])}')
print(f'Single Fetched Sequence Type: {type(col_seq_n)}') # seqrec objects

Number of Sequences stored: 10
List of Sequences Type: <class 'list'>
List Content Type: <class '__main__.SQ'>
Single Fetched Sequence Type: <class '__main__.SQ'>


In [31]:
# Select only a subset as the strand is a little too big (arbitrarily selected)
print('Create subsequence:\n')
subseq_string = col_seq_n.seq[100:500]
col_seq_subset = SQ(seq=subseq_string,
                    seq_type=col_seq_n.seq_type)
col_seq_subset.info()

print('\nDecode DNA: \n')
# get all proteins with a length of more than 1
proteins = col_seq_subset.decode.protein()

# # list all found proteins above a length of 2 
print('Proteins in Sequence col_seq_subset:')
lstcol(proteins,2)

Create subsequence:

SEQ: GACAGTTATGGAAATTAAAATCCTGCACAAGCAGGGAATGAGTAGCCGGGCGATTGCCAGAGAACTGGGGATCTCCCGCAATACCGTTAAACGTTATTTGCAGGCAAAATCTGAGCCGCCAAAATATACGCCGCGACCTGCTGTTGCTTCACTCCTGGATGAATACCGGGATTATATTCGTCAACGCATCGCCGATGCTCATCCTTACAAAATCCCGGCAACGGTAATCGCTCGCGAGATCAGAGACCAGGGATATCGTGGCGGAATGACCATTCTCAGGGCATTCATTCGTTCTCTCTCGGTTCCTCAGGAGCAGGAGCCTGCCGTTCGGTTCGAAACTGAACCCGGACGACAGATGCAGGTTGACTGGGGCACTATGCGTAATGGTCGCTCACCGCTT TYPE: dna

Decode DNA: 

Proteins in Sequence col_seq_subset:
MNALRMVIPPRYPWSLISRAITVAGIL    MLILTKSRQR                     
MVIPPRYPWSLISRAITVAGIL         MP                             
MNTGIIFVNASPMLILTKSRQR         MR                             
MSIGDALTNIIPVFIQE                                             


# <div style="padding: 30px;color:white;margin:10;font-size:80%;text-align:left;display:fill;border-radius:10px;background-color:#F1C40F;overflow:hidden;background-image: url(https://i.imgur.com/tNNAhbu.png)"><b><span style='color:white'>7 |</span></b> Recognising Patterns</div>

- Another important part relating to sequences are pattern recognition within a sequence
- There are specific <b><span style='color:#F1C40F'>patterns</span></b> of nucleotides we can try to find, in order to understand our sequence content a little more
- Having looked at the **<span style='color:#F1C40F'>decoding of proteins</span>** in **Section 4**, we might want to know more about our amino acid chains

For example, an amino acid chain containing the pattern **<span style='color:#F1C40F'>Zinc finger RING-type</span>** | **[Prosite Database](https://prosite.expasy.org/cgi-bin/prosite/prosite-search-ac?PDOC00022)**
- Some proteins known to include a RING finger include **Mammalian breast cancer type 1 susceptibility protein (BRCA1)**
- Let's load the Breast cancer type 1 susceptibility protein **[BRCA1_HUMAN](https://www.uniprot.org/uniprot/P38398#)** in FASTA format (from the **<span style='color:#F1C40F'>UniProtKB</span>** database)
- Using the loader from **Section 5** to see if we have a match as well, since they should be related
- Other **ZnF** types of prosite information **[@github](https://proteinswebteam.github.io/interpro-blog/potm/2007_3/Page1.htm)**

#### <b><span style='color:#F1C40F'>PROSITE PATTERN FORMAT</span></b>
- Looking at the example, it should be quite clear what the below format rules mean, 
- Last point not currently added so you'd need to be specific, which case you're actualy searching for eg. x(2),x(3),x(4) instead of x(2,4)
> - Standard **<span style='color:#F1C40F'>IUPAC amino acid</span>** used to as bases in pattern, separated by -
> - x -> any amino acid acceptable
> - [] -> ambiguity represented by list, any aa in that list acceptable
> - {} -> ambiguity represented by list, any aa other than in {} accepted
> - Repetition of pattern element shown below:
>   x(3) -> x-x-x, x(2,4) -> to x-x or x-x-x or x-x-x-x

In [32]:
# read sequence from file
file_fasta = '/P38398.fasta'
aa_seq = read_seq(file_fasta).get_sq()

READ -> FASTA [generic] | #SEQ: 1


In [33]:
# sequence description
aa_seq.description

'>sp|P38398|BRCA1_HUMAN Breast cancer type 1 susceptibility protein OS=Homo sapiens OX=9606 GN=BRCA1 PE=1 SV=2\n'

In [34]:
# sequence string
aa_seq.seq

'MDLSALRVEEVQNVINAMQKILECPICLELIKEPVSTKCDHIFCKFCMLKLLNQKKGPSQCPLCKNDITKRSLQESTRFSQLVEELLKIICAFQLDTGLEYANSYNFAKKENNSPEHLKDEVSIIQSMGYRNRAKRLLQSEPENPSLQETSLSVQLSNLGTVRTLRTKQRIQPQKTSVYIELGSDSSEDTVNKATYCSVGDQELLQITPQGTRDEISLDSAKKAACEFSETDVTNTEHHQPSNNDLNTTEKRAAERHPEKYQGSSVSNLHVEPCGTNTHASSLQHENSSLLLTKDRMNVEKAEFCNKSKQPGLARSQHNRWAGSKETCNDRRTPSTEKKVDLNADPLCERKEWNKQKLPCSENPRDTEDVPWITLNSSIQKVNEWFSRSDELLGSDDSHDGESESNAKVADVLDVLNEVDEYSGSSEKIDLLASDPHEALICKSERVHSKSVESNIEDKIFGKTYRKKASLPNLSHVTENLIIGAFVTEPQIIQERPLTNKLKRKRRPTSGLHPEDFIKKADLAVQKTPEMINQGTNQTEQNGQVMNITNSGHENKTKGDSIQNEKNPNPIESLEKESAFKTKAEPISSSISNMELELNIHNSKAPKKNRLRRKSSTRHIHALELVVSRNLSPPNCTELQIDSCSSSEEIKKKKYNQMPVRHSRNLQLMEGKEPATGAKKSNKPNEQTSKRHDSDTFPELKLTNAPGSFTKCSNTSELKEFVNPSLPREEKEEKLETVKVSNNAEDPKDLMLSGERVLQTERSVESSSISLVPGTDYGTQESISLLEVSTLGKAKTEPNKCVSQCAAFENPKGLIHGCSKDNRNDTEGFKYPLGHEVNHSRETSIEMEESELDAQYLQNTFKVSKRQSFAPFSNPGNAEEECATFSAHSGSLKKQSPKVTFECEQKEENQGKNESNIKPVQTVNITAGFPVVGQKDKPVDNAKCSIKGGSRFCLSSQFRGNETGLITPNKHGLLQNPYRIPPLFPIKSFVKTKCKKNLL

In [35]:
# find_id -> 'first' (find the first match index)
# find id -> 'all' (find all non overlapping indicies)
# find_id -> 'overlap' (find all overlapping indicies)

aa_seq.pattern.find(pattern='C-x-H-x-[LIVMFY]-C-x(2)-C-[LIVMYA]',
                    find_id='first',                                            
                    search_id='prosite')

showing first for C.H.[LIVMFY]C.{2}C[LIVMYA]


38

#### <b><span style='color:#F1C40F'>STANDARD ABC PATTERNS</span></b>
- We can use standard patterns of nucleotide/amino acid subsequence as well, for example three character pattern: **EES**

In [36]:
# find all EES patterns
aa_seq.pattern.find(pattern='EES',
                    find_id='all',                                            
                    search_id='standard')

found 3 matches


[847, 1539, 1628]

# <div style="padding: 30px;color:white;margin:10;font-size:80%;text-align:left;display:fill;border-radius:10px;background-color:#F1C40F;overflow:hidden;background-image: url(https://i.imgur.com/tNNAhbu.png)"><b><span style='color:white'>8 |</span></b> Creating Subsequences</div>

<div style="color:white;display:fill;border-radius:8px;
            background-color:#03112A;font-size:150%;
            letter-spacing:1.0px">
    <p style="padding: 8px;color:white;"><b><b><span style='color:#F1A424'>8.1 |</span></b> Creating Subsequences</b></p>
</div>

- Our entire sequence may be very long & we might have want cut the current sequence for a variety of purposes
- Example applications:
> - Where a **full sequence** was cut into non overlapping unitigs - **[Identifying Antibiotic Resistant Bacteria](https://www.kaggle.com/shtrausslearning/identifying-antibiotic-resistant-bacteria)**
> - Where a **full sequence** was cut into a predefined sets of nucleotide bases & OHE was used - **[Transcription Factor Binding Location Prediction](https://www.kaggle.com/shtrausslearning/transcription-factor-binding-location-prediction)**
> - <b>Restriction Enzymes</b> can cut DNA sequences at particular segments of the DNA, defined by a particular pattern (we'll look at this here)

<div style="color:white;display:fill;border-radius:8px;
            background-color:#03112A;font-size:150%;
            letter-spacing:1.0px">
    <p style="padding: 8px;color:white;"><b><b><span style='color:#F1A424'>8.2 |</span></b> Dividing Sequences into Parts</b></p>
</div>

#### <b><span style='color:#F1C40F'>CREATING LISTS OF SUBSEQUENCES</span></b>
- Whatever the mechanism, the function <code>cut_pattern</code> is general, requiring us to specify where in the sequence a cut occurs using "|"

In [37]:
# Arbitrary cut pattern at |
n_seq = SQ('AAGATTCGAGCATGCAAACCGGATACA',seq_type='dna')
n_seq.cut.cut_pattern('AGC|AT')

['AAGATTCGAGC', 'ATGCAAACCGGATACA']

<div style="color:white;display:fill;border-radius:8px;
            background-color:#03112A;font-size:150%;
            letter-spacing:1.0px">
    <p style="padding: 8px;color:white;"><b><b><span style='color:#F1A424'>8.3 |</span></b> Generating OHE DNA Features</b></p>
</div>

- We can cut the sequence at a starting location & number of characters that must be present in each sequence
- characters which aren't sufficient to create another segment; the leftover characters are discarded 

In [38]:
# define sequence and create OHE features
seqn = SQ('ATGACGGATCAGCCGCAAGCGGAATTGGCGTTTACGTACGATGCGCCGTAA',seq_type='dna')
features, index = seqn.cut.dnaseq_features(0,10,'dna_features')
features

[note] cutting @0 w/ 10 segments
Input Sequence Length: 51
# Bases cut-off: 1
Updated sequence length: 50
# Segments: 5 created


array([[[0, 0, 1, 0],
        [0, 0, 1, 0],
        [1, 0, 0, 0],
        [1, 0, 0, 0],
        [0, 0, 0, 1],
        [0, 0, 0, 1],
        [0, 0, 1, 0],
        [0, 0, 1, 0],
        [0, 1, 0, 0],
        [0, 0, 1, 0]],

       [[1, 0, 0, 0],
        [0, 0, 1, 0],
        [0, 1, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 1, 0],
        [0, 1, 0, 0],
        [1, 0, 0, 0],
        [1, 0, 0, 0],
        [0, 0, 1, 0],
        [0, 1, 0, 0]],

       [[1, 0, 0, 0],
        [0, 0, 0, 1],
        [0, 0, 1, 0],
        [1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 1, 0],
        [1, 0, 0, 0],
        [0, 0, 0, 1],
        [0, 1, 0, 0]]])

In [39]:
# show index of cut segments
index

['dna_features_0:10', 'dna_features_10:20', 'dna_features_20:30']

# <div style="padding: 30px;color:white;margin:10;font-size:80%;text-align:left;display:fill;border-radius:10px;background-color:#F1C40F;overflow:hidden;background-image: url(https://i.imgur.com/tNNAhbu.png)"><b><span style='color:white'>9 |</span></b> Covid-19 : Protein Identification</div>

<div style="color:white;display:fill;border-radius:8px;
            background-color:#03112A;font-size:150%;
            letter-spacing:1.0px">
    <p style="padding: 8px;color:white;"><b><b><span style='color:#F1A424'>9.1 |</span></b> Section Overview</b></p>
</div>

- Having defined a class that can **<span style='color:#F1C40F'>read</span>**, **<span style='color:#F1C40F'>store</span>** (**<mark style="background-color:#393939;color:white;border-radius:5px;opacity:1.0">read_seq</mark>**) & **<span style='color:#F1C40F'>derive proteins</span>** (**<mark style="background-color:#393939;color:white;border-radius:5px;opacity:1.0">SQ</mark>**) from a sequence file in the **FASTA** format:
- We can read a **sequence from a database** and **store it in our sequence class** & **derive proteins/amino acid chains** from the DNA sequence
- Once we have derived our amino acid chains, **we will want to identify them**, so we need another class that will be able to search a database and match our derived sequence to the ones found inside the database, which we will be able to do using **<mark style="background-color:#393939;color:white;border-radius:5px;opacity:1.0">BLASTwww</mark>**

#### <b><span style='color:#F1C40F'>BLAST - DATABASE SEARCHING</span></b>

Steps we'll be taking:
> - (1) Get the <b><span style='color:#F1C40F'>proteins</span></b> encoded in the genome ( using <code>.translate</code> )
> - (2) Identify the particular protein present & pick some that interest us
> - (3) Using a **local sequence alignmnent** tool called **[BLAST](https://blast.ncbi.nlm.nih.gov/Blast.cgi)**; (using class <code>BLASTwww</code>)
> - (3) Visualise the alignment, so we can see which parts of the sequence match to that of the database


- We will look into <b>Local Sequence Alignment</b> in another notebook **[Biological Sequence Alignment](https://www.kaggle.com/shtrausslearning/custom-class-biological-sequence-alignment)**, which BLAST uses.
- However for the time being keeping things simple; a protein sequence is compared to others already found in a database and returns very simular sequences

<div style="color:white;display:fill;border-radius:8px;
            background-color:#03112A;font-size:150%;
            letter-spacing:1.0px">
    <p style="padding: 8px;color:white;"><b><b><span style='color:#F1A424'>9.2 |</span></b> Reading Sequence from Databses</b></p>
</div>

#### <b><span style='color:#F1C40F'>USING THE NCBI API (via Biopython)</span></b>

To use the class function <code>BLASTwww</code>, we'll first need to:
> - (1) decode the proteins from the DNA sequence 
> - (2) select a specific protein & search for matching results in the database

- <code>BLASTwww</code> - Uses <code>SearchIO.read</code> in Biopython, which enables us to visualise the alignment using <code>view</code>

In [40]:
# Class to access BioPython's blastwww & plot alignments 
class BLASTwww:
    
    # Constructor
    def __init__(self,seq=None,        # input sequence (Seq/string)
                      program='blastp', # BLAST program (blastn/blastp)
                      database='pdb',  # BLAST query database
                      verbose=False,   # write outputs
                      show_top=10,     # maximum number of hits
                      hit_id=0,        # show query match id #
                      read_xml=False,   # read XML BLAST query over new query
                      name=None):      # query identifier
        
        self.seq = seq 
        self.database = database
        self.verbose = verbose
        self.show_top = show_top
        self.hit_id = hit_id
        self.read_xml = read_xml # if present -> read xml
        self.name = name  # search identifier (used for save)
        self.program = program # blast search program
        self.query_df = None   # dataframe that stored query data
        
        # Colourcoding dictionary
        self.dict_cc = {'A':'#3386FF','C':'#3386FF','D':'#B842B2','E':'#B842B2',
                        'F':'#3386FF','G':'#FF5733','H':'#37ADBB','I':'#3386FF',
                        'L':'#3386FF','M':'#3386FF','N':'#24CE5D','P':'#E3E710',
                        'Q':'#24CE5D','R':'#D3385E','S':'#24CE5D','T':'#24CE5D',
                        'V':'#3386FF','W':'#3386FF','Y':'#37ADBB','_':'white',
                        'K':'#D3385E'}
        
    ''' Save BLAST search in CSV format '''
    # read xml & convert to pandas dataframe
    
    def save(self):
        
        # save search xml
        sf = open(f"/blast_{self.name}.xml", "w")
        sf.write(self.__result_handle.read()) 
        self.__result_handle.close()
        sf.close() 

        # save metadata
        self.df.to_csv(f'/csv_{self.name}.csv')
        
    ''' Search for aminoacid chain in database '''
    # Find amino acid sequence in NCBI databases
    # by default, program -> blastp (protein/aa chain search)
    # To prevent multiple fetches, read xml is active & reads xml if fetched
    # more than once
    
    def fetch(self):

        # Input in BioPython Sequence Format
        if(type(self.seq) is str):
            self.seq = Seq(self.seq)
        
        if(self.read_xml == True):
            self.__result_handle = open(f"/blast_{self.name}.xml")
            blast_qresult = SearchIO.read(self.__result_handle,"blast-xml")
        else:
            self.__result_handle = NCBIWWW.qblast(self.program,
                                                  self.database,
                                                  self.seq)
            
            blast_qresult = SearchIO.read(self.__result_handle,"blast-xml")
            self.read_xml = True # blast query has been saved
 
        ii=-1
        lst_id = []; lst_descr = []; lst_bitscore = []
        lst_ali = []; lst_eval = []
        
        for f in blast_qresult: 
            ii+=1
            seqid = blast_qresult[ii]
            details = seqid[0]

            lst_id.append(seqid.id)
            lst_descr.append(seqid.description)
            lst_eval.append(details.evalue)
            lst_bitscore.append(details.bitscore)
            lst_ali.append(details.aln)

        dic_data = {'id':lst_id,'description':lst_descr,
                    'evalue':lst_eval,'bitscore':lst_bitscore,
                    'alignment':lst_ali}

        self.__query_df = pd.DataFrame(dic_data)

        if(self.verbose):

            print(blast_qresult[0:self.show_top])
            #fetch the id, description, evalue, bitscore & alignment
            print(f'\nShowing BLAST query result #{hit_id}')
            print('------------------------------------------------------------')
            seqid = blast_qresult[hit_id]
            details = seqid[hit_id]

            print(f'Sequence ID: {seqid.id}')
            print(f'Description: {seqid.description}')
            print(f'e-value: {details.evalue}')
            print(f'Bit Score: {details.bitscore}')

            print('\nShowing Alignment:')
            print('------------------------------------------------------------')
            print(f"Alignment:\n{details.aln}")
            
    def __get_colors(self,seqs):
        """make colors for bases in sequence"""
        text = [i for s in list(seqs) for i in s]
        colors = [self.dict_cc[i] for i in text]
        return colors
    
    ''' Visualise BLAST query '''
    # After fetch, view result dataframe
    
    def view_query(self):
        
        if(self.__query_df is not None):
            return self.__query_df
            
    ''' View Alignment '''
    # View BLAST alignment using Bokeh
    
    def view_alignment(self,aln_id,
                  fontsize="9pt",
                  plot_width=800):
        
        # Choose Alignment 
        if(self.__query_df is not None):
            aln_id = self.__query_df.loc[aln_id,'alignment']
            
        #make sequence and id lists from the aln object
        seqs = [rec.seq for rec in (aln_id)]
        ids = [rec.id for rec in aln_id]    
        text = [i for s in list(seqs) for i in s]
        colors = self.__get_colors(seqs)    
        N = len(seqs[0])
        S = len(seqs)    

        x = np.arange(0.5,N+0.5)
        y = np.arange(0,S,1)
        #creates a 2D grid of coords from the 1D arrays
        xx, yy = np.meshgrid(x, y)
        #flattens the arrays
        gx = xx.ravel()
        gy = yy.flatten()
        #use recty for rect coords with an offset
        recty = gy+0.5
        h= 1/S
        #now we can create the ColumnDataSource with all the arrays
        source = ColumnDataSource(dict(x=gx, y=gy, recty=recty, text=text, colors=colors))
        plot_height = len(seqs)*15+25
        x_range = Range1d(0,N+1, bounds='auto')
        if N>100:
            viewlen=50
        else:
            viewlen=N
        #view_range is for the close up view
        view_range = (0,viewlen)
        tools="xpan, xwheel_zoom, reset, save" 

        #entire sequence view (no text, with zoom)
        p = figure(title=None, plot_width= plot_width, plot_height=50,
                   x_range=x_range, y_range=(0,S), tools=tools,
                   min_border=0, toolbar_location='below')
        rects = Rect(x="x", y="recty",  width=1, height=1, fill_color="colors",
                     line_color=None, fill_alpha=0.6)
        p.add_glyph(source, rects)
        p.yaxis.visible = False
        p.grid.visible = False  

        #sequence text view with ability to scroll along x axis
        p1 = figure(title=None, plot_width=plot_width, plot_height=plot_height,
                    x_range=view_range, y_range=ids, tools="xpan,reset",
                    min_border=0, toolbar_location='below')#, lod_factor=1)          
        glyph = Text(x="x", y="y", text="text", text_align='center',text_color="black",
                    text_font_size=fontsize,text_font_style='bold')
        rects = Rect(x="x", y="recty",  width=1.0, height=1, fill_color="colors",
                    line_color=None, fill_alpha=0.4)
        p1.add_glyph(source, glyph)
        p1.add_glyph(source, rects)

        p1.grid.visible = False
        p1.xaxis.major_label_text_font_style = "bold"
        p1.yaxis.minor_tick_line_width = 0
        p1.yaxis.major_tick_line_width = 0
        
        return gridplot([[p],[p1]],toolbar_location='below') 

<div style="color:white;display:fill;border-radius:8px;
            background-color:#03112A;font-size:150%;
            letter-spacing:1.0px">
    <p style="padding: 8px;color:white;"><b><b><span style='color:#F1A424'>9.3 |</span></b> Protein Identification</b></p>
</div>

#### <b><span style='color:#F1C40F'>READ SEQUENCE FROM DATABASE</span></b>

- Let's use the **<span style='color:#F1C40F'>coronavirus genomoe/sequence</span>**, already uploaded to **[Kaggle](https://www.kaggle.com/paultimothymooney/coronavirus-genome-sequence)** & **[original source](https://www.ncbi.nlm.nih.gov/nuccore/NC_045512)**

In [41]:
# Read FASTA format containing Covid Genome
virus_fna = '/MN908947.fna'
virus_n = read_seq(virus_fna) # read and store FNA data 
print(f'Sequence Header: {virus_n.lst_header[0]}')

READ -> FASTA [nucleic acid] | #SEQ: 1
Sequence Header: >MN908947.3 Severe acute respiratory syndrome coronavirus 2 isolate Wuhan-Hu-1, complete genome



In [42]:
# Store the sequence, defining a SQ class instance
virus_sq = virus_n.get_sq()
print(f'Sequence length: {len(virus_sq)} nucleotides')

# The sequence is a bit too long, lets show the first 1000 characters
virus_sq[0:500]

Sequence length: 29903 nucleotides


'ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGTAGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTCGGCTGCATGCTTAGTGCACTCACGCAGTATAATTAATAACTAATTACTGTCGTTGACAGGACACGAGTAACTCGTCTATCTTCTGCAGGCTGCTTACGGTTTCGTCCGTGTTGCAGCCGATCATCAGCACATCTAGGTTTCGTCCGGGTGTGACCGAAAGGTAAGATGGAGAGCCTTGTCCCTGGTTTCAACGAGAAAACACACGTCCAACTCAGTTTGCCTGTTTTACAGGTTCGCGACGTGCTCGTACGTGGCTTTGGAGACTCCGTGGAGGAGGTCTTATCAGAGGCACGTCAACATCTTAAAGATGGCACTTGTGGCTTAGTAGAAGTTGAAAAAGGCGTTTTGCCTCAACTTGAACAGCCCTATGTGTTCATCAAACGTTCGGATGCTCGAACTG'

#### <b><span style='color:#F1C40F'>NUCLEOTIDE BASE DISTRIBUTION</span></b>
- We can visualise the nucleotide frequency in the entire sequence & also check the GC content (% wise)

In [43]:
# The virus contains a heavy portion of Thymine & Adenine (63%)
virus_sq.count.freq(fsize=[300,800],
                    title='Nucleotide Content (%)')

print(f'GC-Content Sequence 1: {round(virus_sq.count.gc()*100,3)}%')

GC-Content Sequence 1: 37.97%


#### <b><span style='color:#F1C40F'>DERIVE AMINO ACID CHAINS IN SEQUENCE</span></b>
- Using the <code>get_protein</code> method, we can obtain a list of amino acid chains by translating the sequence

In [44]:
# Get all amino acid chains above default = 1
lst_prot = virus_sq.decode.protein()
print(f'Total Number of Amino Acid Chains: {len(lst_prot)}')

Total Number of Amino Acid Chains: 1208


#### <b><span style='color:#F1C40F'>FUNCTIONAL PROTEINS & OLIGOPEPTIDES</span></b>

- Upon translation, our <b>amino acid chains</b> aren't actually all proteins, some extra criteria these chains must meet in order to be classified as proteins exist: 
> - **<mark style="background-color:#F1C40F;color:white;border-radius:5px;opacity:0.9">Functional proteins</mark>** are chains above <b>20 amino acids</b>.
> - Smaller chains are called **<mark style="background-color:#F1C40F;color:white;border-radius:5px;opacity:0.9">oligopeptides</mark>** (2-20 amino acids) & have other functionalities.
- Let's select the <b>largest amino acid chain</b> found in the genome & search for it in a databse, so we can identify it.
- Interestingly enough, the largest amino acid chain actually differred from the Biopython result I tried in **[that notebook](https://www.kaggle.com/shtrausslearning/biopython-bioinformatics-basics)**

In [45]:
# For convenience, let's select a subset above 50 amino acids
print(f'Amino Acid Chains: {len(virus_sq.decode.protein(mins=50))}')
largest_aa = virus_sq.decode.protein(mins=50)[0]
print(f'Largest Amino Acid: {largest_aa}')
print(f'Length of Largest Amino Acid: {len(virus_sq.decode.protein(mins=50)[0])}')

Amino Acid Chains: 243
Largest Amino Acid: MESLVPGFNEKTHVQLSLPVLQVRDVLVRGFGDSVEEVLSEARQHLKDGTCGLVEVEKGVLPQLEQPYVFIKRSDARTAPHGHVMVELVAELEGIQYGRSGETLGVLVPHVGEIPVAYRKVLLRKNGNKGAGGHSYGADLKSFDLGDELGTDPYEDFQENWNTKHSSGVTRELMRELNGGAYTRYVDNNFCGPDGYPLECIKDLLARAGKASCTLSEQLDFIDTKRGVYCCREHEHEIAWYTERSEKSYELQTPFEIKLAKKFDTFNGECPNFVFPLNSIIKTIQPRVEKKKLDGFMGRIRSVYPVASPNECNQMCLSTLMKCDHCGETSWQTGDFVKATCEFCGTENLTKEGATTCGYLPQNAVVKIYCPACHNSEVGPEHSLAEYHNESGLKTILRKGGRTIAFGGCVFSYVGCHNKCAYWVPRASANIGCNHTGVVGEGSEGLNDNLLEILQKEKVNINIVGDFKLNEEIAIILASFSASTSAFVETVKGLDYKAFKQIVESCGNFKVTKGKAKKGAWNIGEQKSILSPLYAFASEAARVVRSIFSRTLETAQNSVRVLQKAAITILDGISQYSLRLIDAMMFTSDLATNNLVVMAYITGGVVQLTSQWLTNIFGTVYEKLKPVLDWLEEKFKEGVEFLRDGWEIVKFISTCACEIVGGQIVTCAKEIKESVQTFFKLVNKFLALCADSIIIGGAKLKALNLGETFVTHSKGLYRKCVKSREETGLLMPLKAPKEIIFLEGETLPTEVLTEEVVLKTGDLQPLEQPTSEAVEAPLVGTPVCINGLMLLEIKDTEKYCALAPNMMVTNNTFTLKGGAPTKVTFGDDTVIEVQGYKSVNITFELDERIDKVLNEKCSAYTVELGTEVNEFACVVADAVIKTLQPVSELLTPLGIDLDEWSMATYYLFDESGEFKLASHMYCSFYPPDEDEEEGDCEEEEFEPSTQYEYGTEDDYQG

#### <b><span style='color:#F1C40F'>SEARCH DATABASE</span></b>
- Let's search for one of the chains found in the list <code>lst_prot</code>, <code>largest_aa</code> using **<mark style="background-color:#393939;color:white;border-radius:5px;opacity:1.0">BLASTwww</mark>**
- First we initialise class **<mark style="background-color:#393939;color:white;border-radius:5px;opacity:1.0">BLASTwww</mark>**, at minimum providing the sequence which we want to find as an argument
- The **BLAST** query is then carried our when the <code>fetch</code> method is called
- The results are stored in method <code>view_query</code>, which contain the **<mark style="background-color:#393939;color:white;border-radius:5px;opacity:1.0">BioPython</mark>** format **alignment** data as well

In [46]:
blast_query = BLASTwww(largest_aa,name='c19')
blast_query.fetch()
blast_query.view_query()

,id,description,evalue,bitscore,alignment
0,pdb|7MSW|A,"Chain A, Non-structural protein 2 [Severe acut...",0.0,1328.540,"((A, Y, T, R, Y, V, D, N, N, F, C, G, P, D, G,..."
1,pdb|6WUU|A,"Chain A, Non-structural protein 3 [Severe acut...",0.0,674.855,"((L, R, E, V, R, T, I, K, V, F, T, T, V, D, N,..."
2,pdb|7CMD|A,"Chain A, Non-structural protein 3 [Severe acut...",0.0,674.470,"((E, V, R, T, I, K, V, F, T, T, V, D, N, I, N,..."
3,pdb|7CJD|A,"Chain A, Non-structural protein 3 [Severe acut...",0.0,671.389,"((E, V, R, T, I, K, V, F, T, T, V, D, N, I, N,..."
4,pdb|6XAA|A,"Chain A, Non-structural protein 3 [Severe acut...",0.0,670.618,"((R, E, V, R, T, I, K, V, F, T, T, V, D, N, I,..."
5,pdb|6XA9|A,"Chain A, Non-structural protein 3 [Severe acut...",0.0,669.463,"((R, E, V, R, T, I, K, V, F, T, T, V, D, N, I,..."
6,pdb|7D47|A,"Chain A, Non-structural protein 3 [Severe acut...",0.0,669.078,"((E, V, R, T, I, K, V, F, T, T, V, D, N, I, N,..."
7,pdb|6W9C|A,"Chain A, Non-structural protein 3 [Severe acut...",0.0,668.692,"((E, V, R, T, I, K, V, F, T, T, V, D, N, I, N,..."
8,pdb|7NT4|A,"Chain A, Non-structural protein 3 [Severe acut...",0.0,668.307,"((E, V, R, T, I, K, V, F, T, T, V, D, N, I, N,..."
9,pdb|6WZU|A,"Chain A, Non-structural protein 3 [Severe acut...",0.0,668.307,"((E, V, R, T, I, K, V, F, T, T, V, D, N, I, N,..."


- Next, let's choose a few search hit results & plot them, visualising the results: <code>pdb|7MSW|A</code>, <code>pdb|6WUU|A</code>
- If we have a hit, we of course can search & find more about this specific sequence in literature or databases
- We can see that our perfect hit case; <b>pdb|7WUU|A</b> is not the entire sequence that we input & visually can see its much smaller than our input
- We got a match for the the entire sequence, using the <b>UniProt database</b>. 
- However as we can see with the <b>pdb</b>, we only managed to identify sequences representing part of the protein, which still gave us 100% match locally.

In [47]:
# Visualise the highest up on the list hit
plot = blast_query.view_alignment(0) # view first alignment (index=0) 
pn.pane.Bokeh(plot)                  # plot using bokeh

Bokeh(Column)

- We can obtain the individual sequences from the **alignment** in the following way
- Using the <code>find</code> method, we can find where in out input sequence we actually got a hit

In [48]:
blast_df = blast_query.view_query()
align0 = blast_df.loc[0,'alignment']

print('Search hit sequence:')
print(f'length of hit sequence: {len(align0[1].seq)}','\n')
print(align0[1].seq)

as_seq = SQ(largest_aa)
aa_seq.pattern.find(pattern=str(align0[1].seq),
                    find_id='all',                                            
                    search_id='standard')

Search hit sequence:
length of hit sequence: 638 

AYTRYVDNNFCGPDGYPLECIKDLLARAGKASCTLSEQLDFIDTKRGVYCCREHEHEIAWYTERSEKSYELQTPFEIKLAKKFDTFNGECPNFVFPLNSIIKTIQPRVEKKKLDGFMGRIRSVYPVASPNECNQMCLSTLMKCDHCGETSWQTGDFVKATCEFCGTENLTKEGATTCGYLPQNAVVKIYCPACHNSEVGPEHSLAEYHNESGLKTILRKGGRTIAFGGCVFSYVGCHNKCAYWVPRASANIGCNHTGVVGEGSEGLNDNLLEILQKEKVNINIVGDFKLNEEIAIILASFSASTSAFVETVKGLDYKAFKQIVESCGNFKVTKGKAKKGAWNIGEQKSILSPLYAFASEAARVVRSIFSRTLETAQNSVRVLQKAAITILDGISQYSLRLIDAMMFTSDLATNNLVVMAYITGGVVQLTSQWLTNIFGTVYEKLKPVLDWLEEKFKEGVEFLRDGWEIVKFISTCACEIVGGQIVTCAKEIKESVQTFFKLVNKFLALCADSIIIGGAKLKALNLGETFVTHSKGLYRKCVKSREETGLLMPLKAPKEIIFLEGETLPTEVLTEEVVLKTGDLQPLEQPTSEAVEAPLVGTPVCINGLMLLEIKDTEKYCALAPNMMVTNNTFTLKGG
no matches for AYTRYVDNNFCGPDGYPLECIKDLLARAGKASCTLSEQLDFIDTKRGVYCCREHEHEIAWYTERSEKSYELQTPFEIKLAKKFDTFNGECPNFVFPLNSIIKTIQPRVEKKKLDGFMGRIRSVYPVASPNECNQMCLSTLMKCDHCGETSWQTGDFVKATCEFCGTENLTKEGATTCGYLPQNAVVKIYCPACHNSEVGPEHSLAEYHNESGLKTILRKGGRTIAFGGCVFSYVGCHNKCAYWVPRASANIGCNHTGVVGEGSEGLNDNLLEILQKEKVNINIVGDFKLNEEIAI

In [49]:
# blast_query.save() # save metadata (alignments arent actually saved)
# to save alignments, just save xml of BLAST search

<div style="color:white;display:fill;border-radius:8px;
            background-color:#03112A;font-size:150%;
            letter-spacing:1.0px">
    <p style="padding: 8px;color:white;"><b><b><span style='color:#F1A424'>9.4 |</span></b> Manually Searching Other Databases</b></p>
</div>

- There are quite a few databses, we only checked the **pdb** database & **not all are accessible** via the NCBI API
- We might not actually find a match for the entire sequence all the time, especially if it's as long as the one we have, naturally it will likely contain different subsequences
- We can use this string; copy & paste this protein & search for it using the [PSI BLAST](https://www.ebi.ac.uk/Tools/sss/psiblast/) in order to find proteins already in a predefined database. 
- For the protein sequence that we searched for:
> The results indicate that a 100% match was found with an already existing sequence in the database: **[sp|P0C6U8|R1A_SARS Replicase polyprotein 1a](https://pubchem.ncbi.nlm.nih.gov/protein/P0C6U8)**


What is a **<mark style="background-color:#F1C40F;color:white;border-radius:5px;opacity:0.9">polyprotein</mark>** | **[NCBI](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7125721/)**
> Polyproteins are chains of covalently conjoined smaller proteins that occur in nature as versatile means to organize the proteome of viruses including HIV.

# <div style="padding: 30px;color:white;margin:10;font-size:80%;text-align:left;display:fill;border-radius:10px;background-color:#F1C40F;overflow:hidden;background-image: url(https://i.imgur.com/tNNAhbu.png)"><b><span style='color:white'>10 |</span></b> Summary</div>

#### <b><span style='color:#F1C40F'>CUSTOM CLASSES</span></b>
- In this notebook, we have defined a class that stores <b>basic information</b> about a specific sequence of interest. 
- Another class was also defined in order to read a commonly encountered format type <b>FASTA</b>. 

#### <b><span style='color:#F1C40F'>OVERVIEW</span></b>
- We also looked at common sequence related operations such as <b>transcription</b> & <b>translation</b>, which are quite critical concepts.
- We also looked at some basic visualisation of the nucleotide & amino acid distributions, which can be helpful when comparing not only the <b>count</b>, but also the percentage of each alphabet for the corresponding sequence type.

### <b><span style='color:#F1C40F'>WHERE TO FROM HERE</span></b>
- As with the <b>BioPython</b> module, we have defined classes which store sequence data & which can be expanded further.
- The class, <b>SQ</b> will be used as a basis for other sequence related operations, such as <b>pairwise</b> & <b>multiple sequence alignment</b>, which due to their content size alone, can have their own dedicated classes, similar to the class <b>READ_SEQ</b>, which is dedicated to reading and passing on the sequence information to class <b>SQ</b>.
- The notebook will of course be continuously updated and futher functions will be added in the future.